**O QUE É A RAIS?**

A RAIS é uma base de dados estatística gerada a partir da declaração da RAIS (Relação Anual de Informações Sociais) instituída para auxiliar a gestão governamental do setor do trabalho com base legal nos Decreto nº 76.900, de 23 de dezembro de 1975 e nº 10.854, de 10 de novembro de 2021. A partir da coleta, o governo visa:

1. o suprimento às necessidades de controle da atividade trabalhista no País,
2. o provimento de dados para a elaboração de estatísticas do trabalho,
3. a disponibilização de informações do mercado de trabalho às entidades governamentais,
4. embasar estudos técnicos de natureza estatística e atuarial;
5. suprir outras necessidades relacionadas ao controle de registros do FGTS, dos Sistemas de Arrecadação e de Concessão e Benefícios Previdenciários, e da identificação do trabalhador com direito ao abono salarial PIS/PASEP (APENAS RAIS IDENTIFICADA).

Atualmente, duas bases de dados são disponibilizadas anualmente de forma gratuita pela internet, a RAIS Trabalhadores (também conhecida como RAIS Vínculos) e a RAIS Estabelecimentos (período de 1985 - ano atual).

A base RAIS Trabalhadores é organizada em nível do vínculo, contém todos os vínculos declarados (ativos e não ativos em 31/12). Abaixo algumas informações que podem ser obtidas a partir do uso desta base:

Número de empregados em 31 de dezembro, segundo faixa etária, escolaridade e gênero por nível geográfico, setorial e ocupacional;
Número de empregados por tamanho de estabelecimento, segundo setor de atividade econômica;
Remuneração média dos empregos em 31 de dezembro, segundo ocupação e setor de atividade econômica por nível geográfico.

A base RAIS Estabelecimentos é organizada em nível do estabelecimento. Contém tanto os estabelecimentos com vínculo declarado quanto os estabelecimentos sem vínculo informado no exercício (RAIS negativa). Abaixo informações que podem ser obtidas a partir desta base:

Freqüência (quantidade) de estabelecimentos declarantes
Informações do Estoque (quantidade de empregos em 31/12)
Nível geográfico;
Nível setorial;
Natureza jurídica;
Tamanho do estabelecimento;
Indicador de RAIS Negativa.

Fonte das informações: http://acesso.mte.gov.br/portal-pdet/o-pdet/portifolio-de-produtos/bases-de-dados.htm e http://www.rais.gov.br/sitio/sobre.jsf   

**O QUE ESSE CÓDIGO FAZ?**

Esse código faz o download, tratamento e o carregamento dos dados da RAIS estabelecimentos para o mongodb e para o postgresql no Dbeaver.  


# **1. IMPORTANDO LIBS**:

In [1]:
print("Importando bibliotecas necessárias...")

from datetime import date #manipulação de dados do tipo data/hora
from datetime import datetime #manipulação de dados do tipo data/hora
from ftplib import FTP #conexão ftp
import json
import os #manipulação de arquivos/diretórios
from os import walk #manipulação de arquivos/diretórios
#import psycopg2 as p
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from datetime import date
from pyspark.sql.functions import col, when, trim, length, concat_ws, expr, lpad, udf
from pyspark.sql.types import StringType, IntegerType
import py7zr #manipulação de arquivos zipados com 7zip
import requests #requisições na internet
import requests_ftp #requisições ftp

#os.environ['SPARK_HOME'] = r'C:\spark'
#os.environ['JAVA_HOME'] = r'C:\jdk1.8.0_202'

print("\nBibliotecas importadas, bora para a execução do script!!")

Importando bibliotecas necessárias...

Bibliotecas importadas, bora para a execução do script!!


# **2. DEFININDO DIR. P/ ARMAZENAMENTO TEMPORÁRIO DOS DADOS**:

In [2]:
base = input('Insira o caminho no qual os dados serão temporariamente armazenados antes de os enviarmos ao nosso banco de dados Mongodb. A título de exemplo, veja como está no computador do Marcilio: "E:\\Users\\c2705\\PROJETOS\\AUTOMATIZACAO\\RAIS\\ESTABELECIMENTOS\\04_temp_data"\n')
print("\nObrigado! Os arquivos serão armazenados temporariamente neste diretório da sua máquina: ", base, "\n")
#caminho_reports = input('Agora insira o caminho onde você irá armazenar os relatórios de Pd Profilling, se necessário. No caso do Marcilio, ficou da seguinte forma: "E:\\Users\\c2705\\PROJETOS\\AUTOMATIZACAO\\RAIS\\ESTABELECIMENTOS\\04_temp_data\\0401_data_reports"')
#print("\nObrigado! Os relatórios serão armazenados temporariamente neste diretório da sua máquina: ", caminho_reports, "\n")


Obrigado! Os arquivos serão armazenados temporariamente neste diretório da sua máquina:  E:\Users\c2705\PROJETOS\AUTOMATIZACAO\git\automatizacao-etls\RAIS\ESTABELECIMENTOS\04_temp_data 



# **3. SETANDO FTP**:

In [3]:
# Portal principal da RAIS
ftp = FTP("ftp.mtps.gov.br")  ## SERVIDOR FTP
link = r"ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/"

In [4]:
# Comando que conecta a aba correta dentro do site ftp
cmd = r"pdet/microdados/RAIS/"
print(cmd) 

pdet/microdados/RAIS/


In [5]:
# Iniciando sessão no ftp e armazenando variável de requisição
requests_ftp.monkeypatch_session()
s = requests.Session()

In [6]:
# Conferindo se os diretórios dos anos escolhidos para download do ftp existem
anos = []
while True:
    try:
        ano_inicial = int(input('Digite o ANO INICIAL para o qual você deseja fazer o ETL da RAIS: '))
        ano_final = int(input('Digite o ÚLTIMO ANO para o qual você deseja fazer o ETL da RAIS: '))
        if len(str(ano_inicial)) == 4 and str(ano_inicial).startswith("20") and len(str(ano_final)) == 4 and str(ano_final).startswith("20"):
            for i in range(ano_inicial, ano_final+1):
                resp = s.list(link+str(i))
                code = resp.status_code
                if code == 226:
                    anos.append(str(i))
                    print(f"Opaa, que beleza!! Encontramos o diretório do ano de {i} no ftp.")
                else:
                    print(f"Eita, não encontramos o diretório do ano de {i} no ftp :S.")
            print("\nA lista final dos anos de referência para o ETL dos dados contém os seguintes anos: ", anos)
            break
        else:
            print("Erro: Por favor, insira anos válidos com quatro dígitos começando com '20', ex: 2022.")
    except ValueError:
        print("Erro: Por favor, insira um número inteiro, ex: 2022.")

Opaa, que beleza!! Encontramos o diretório do ano de 2023 no ftp.

A lista final dos anos de referência para o ETL dos dados contém os seguintes anos:  ['2023']


# **4. CRIANDO FUNÇÕES**:

In [7]:
# Criando função pra checar se já existe o arquivo no diretório

def checadir(pasta, arquivo):
    files = []
    for filenames in walk(pasta):
        files.extend(filenames)
        break
    if arquivo in files:
        print(f"O arquivo {arquivo} já existe. Indo para o próximo!\n")
        return True
    else:
        return False

In [8]:
# Criando função para fazer o download dos arquivos
def baixar_arquivo(ftp, nome_arquivo, pasta_local):
    print("\nBaixando o arquivo do ftp...\n")
    with open(os.path.join(pasta_local, nome_arquivo), "wb") as fp:
        ftp.retrbinary("RETR " + nome_arquivo, fp.write)
    print(f'Arquivo {nome_arquivo} baixado com sucesso!\n')

In [9]:
def extrair_arquivo_7z_txt(caminho_arquivo):
    with py7zr.SevenZipFile(caminho_arquivo, mode="r") as z:
        z.extractall(path=base)
    print(f"Arquivo txt extraído do 7zip para o caminho: {caminho_arquivo} com sucesso!\n")

In [10]:
# Função para remover arquivos
def remover_arquivo_7z(caminho_arquivo):
    os.remove(caminho_arquivo)
    print(f"Arquivo 7zip removido do caminho: {caminho_arquivo} com sucesso!\n")

In [11]:
# Dicionários de padronização sebrae

# Classificação do Sebrae sobre o porte da empresa com base no setor do IBGE
pad_sebrae_porte = {
"Agropecuária, Extração Vegetal, Caça e Pesca1":"M/P",
"Agropecuária, Extração Vegetal, Caça e Pesca10":"M/G",
"Agropecuária, Extração Vegetal, Caça e Pesca2":"M/P",
"Agropecuária, Extração Vegetal, Caça e Pesca4":"M/P",
"Agropecuária, Extração Vegetal, Caça e Pesca7":"M/G",
"Agropecuária, Extração Vegetal, Caça e Pesca5":"M/P",
"Agropecuária, Extração Vegetal, Caça e Pesca8":"M/G",
"Agropecuária, Extração Vegetal, Caça e Pesca3":"M/P",
"Agropecuária, Extração Vegetal, Caça e Pesca6":"M/G",
"Agropecuária, Extração Vegetal, Caça e Pesca9":"M/G",
"Comércio1":"Micro",
"Comércio10":"Grande",
"Comércio2":"Micro",
"Comércio4":"Pequena",
"Comércio7":"Grande",
"Comércio5":"Pequena",
"Comércio8":"Grande",
"Comércio3":"Micro",
"Comércio6":"Média",
"Comércio9":"Grande",
"Construção Civil1":"Micro",
"Construção Civil10":"Grande",
"Construção Civil2":"Micro",
"Construção Civil4":"Micro",
"Construção Civil7":"Média",
"Construção Civil5":"Pequena",
"Construção Civil8":"Média",
"Construção Civil3":"Micro",
"Construção Civil6":"Pequena",
"Construção Civil9":"Grande",
"Extrativa Mineral1":"Micro",
"Extrativa Mineral10":"Grande",
"Extrativa Mineral2":"Micro",
"Extrativa Mineral4":"Micro",
"Extrativa Mineral7":"Média",
"Extrativa Mineral5":"Pequena",
"Extrativa Mineral8":"Média",
"Extrativa Mineral3":"Micro",
"Extrativa Mineral6":"Pequena",
"Extrativa Mineral9":"Grande",
"Indústria de Transformação1":"Micro",
"Indústria de Transformação10":"Grande",
"Indústria de Transformação2":"Micro",
"Indústria de Transformação4":"Micro",
"Indústria de Transformação7":"Média",
"Indústria de Transformação5":"Pequena",
"Indústria de Transformação8":"Média",
"Indústria de Transformação3":"Micro",
"Indústria de Transformação6":"Pequena",
"Indústria de Transformação9":"Grande",
"Serviços Industriais de Utilidade Pública1":"Micro",
"Serviços Industriais de Utilidade Pública10":"Grande",
"Serviços Industriais de Utilidade Pública2":"Micro",
"Serviços Industriais de Utilidade Pública4":"Micro",
"Serviços Industriais de Utilidade Pública7":"Média",
"Serviços Industriais de Utilidade Pública5":"Pequena",
"Serviços Industriais de Utilidade Pública8":"Média",
"Serviços Industriais de Utilidade Pública3":"Micro",
"Serviços Industriais de Utilidade Pública6":"Pequena",
"Serviços Industriais de Utilidade Pública9":"Grande",
"Serviços1":"Micro",
"Serviços10":"Grande",
"Serviços2":"Micro",
"Serviços4":"Pequena",
"Serviços7":"Grande",
"Serviços5":"Pequena",
"Serviços8":"Grande",
"Serviços3":"Micro",
"Serviços6":"Média",
"Serviços9":"Grande",
"Administração Pública1":"Não se aplica",
"Administração Pública10":"Não se aplica",
"Administração Pública2":"Não se aplica",
"Administração Pública4":"Não se aplica",
"Administração Pública7":"Não se aplica",
"Administração Pública5":"Não se aplica",
"Administração Pública8":"Não se aplica",
"Administração Pública3":"Não se aplica",
"Administração Pública6":"Não se aplica",
"Administração Pública9":"Não se aplica",
"-11":"-1",
"-12":"-1",
"-13":"-1",
"-14":"-1",
"-15":"-1",
"-16":"-1",
"-17":"-1",
"-18":"-1",
"-19":"-1",
"-110":"-1",
"-1-1":"-1",
"Administração Pública-1":"Não se aplica",
"Serviços Industriais de Utilidade Pública-1":"-1",
"Serviços-1":"-1",
"Extrativa Mineral-1":"-1",
"Indústria de Transformação-1":"-1",
"Comércio-1":"-1",
"Construção Civil-1":"-1",
"Agropecuária, Extração Vegetal, Caça e Pesca-1":"-1"
}

# Classificação do Sebrae sobre o setor da empresa com base na CNAE 2 SUBCLASSE
pad_sebrae_setor = {
'0112101':'Agropecuária',
'0500301':'Indústria',
'0892401':'Indústria',
'1052000':'Indústria',
'1210700':'Indústria',
'0112102':'Agropecuária',
'2532202':'Indústria',
'2814301':'Indústria',
'2920401':'Indústria',
'3102100':'Indústria',
'4940000':'Serviços',
'0112199':'Agropecuária',
'0113000':'Agropecuária',
'0114800':'Agropecuária',
'0115600':'Agropecuária',
'0116401':'Agropecuária',
'0116402':'Agropecuária',
'0116403':'Agropecuária',
'8599603':'Serviços',
'4723700':'Comércio',
'4911600':'Serviços',
'4930203':'Serviços',
'0116499':'Agropecuária',
'0119901':'Agropecuária',
'0119902':'Agropecuária',
'5612100':'Serviços',
'6010100':'Serviços',
'6493000':'Serviços',
'0119903':'Agropecuária',
'0119904':'Agropecuária',
'9603301':'Serviços',
'0119905':'Agropecuária',
'0119906':'Agropecuária',
'0119907':'Agropecuária',
'0119908':'Agropecuária',
'0119909':'Agropecuária',
'0119999':'Agropecuária',
'0121101':'Agropecuária',
'0121102':'Agropecuária',
'0122900':'Agropecuária',
'0131800':'Agropecuária',
'0132600':'Agropecuária',
'0133401':'Agropecuária',
'0133402':'Agropecuária',
'9700500':'Serviços',
'0133403':'Agropecuária',
'0133404':'Agropecuária',
'0133405':'Agropecuária',
'0133406':'Agropecuária',
'0133407':'Agropecuária',
'0133408':'Agropecuária',
'0133409':'Agropecuária',
'0133410':'Agropecuária',
'0133411':'Agropecuária',
'0133499':'Agropecuária',
'0134200':'Agropecuária',
'0135100':'Agropecuária',
'0151201':'Agropecuária',
'0139301':'Agropecuária',
'0139302':'Agropecuária',
'0139303':'Agropecuária',
'0139304':'Agropecuária',
'0139305':'Agropecuária',
'0139306':'Agropecuária',
'0139399':'Agropecuária',
'0141501':'Agropecuária',
'0141502':'Agropecuária',
'0142300':'Agropecuária',
'0151202':'Agropecuária',
'0151203':'Agropecuária',
'0152101':'Agropecuária',
'0152102':'Agropecuária',
'0152103':'Agropecuária',
'0153901':'Agropecuária',
'0153902':'Agropecuária',
'0154700':'Agropecuária',
'0155501':'Agropecuária',
'0155502':'Agropecuária',
'0155503':'Agropecuária',
'0155504':'Agropecuária',
'0155505':'Agropecuária',
'0159801':'Agropecuária',
'0159802':'Agropecuária',
'0159803':'Agropecuária',
'0159804':'Agropecuária',
'0159899':'Agropecuária',
'0161001':'Agropecuária',
'0161002':'Agropecuária',
'0161003':'Agropecuária',
'0161099':'Agropecuária',
'0162801':'Agropecuária',
'0162802':'Agropecuária',
'0162803':'Agropecuária',
'0162899':'Agropecuária',
'0163600':'Agropecuária',
'0170900':'Agropecuária',
'0210101':'Agropecuária',
'0210102':'Agropecuária',
'0210103':'Agropecuária',
'0210104':'Agropecuária',
'0210105':'Agropecuária',
'0210106':'Agropecuária',
'0210107':'Agropecuária',
'0210108':'Agropecuária',
'0210109':'Agropecuária',
'0210199':'Agropecuária',
'0220901':'Agropecuária',
'0220902':'Agropecuária',
'0220903':'Agropecuária',
'0220904':'Agropecuária',
'0220905':'Agropecuária',
'0220906':'Agropecuária',
'0220999':'Agropecuária',
'0230600':'Serviços',
'0311601':'Agropecuária',
'0311602':'Agropecuária',
'0311603':'Agropecuária',
'0311604':'Agropecuária',
'0312401':'Agropecuária',
'0312402':'Agropecuária',
'0312403':'Agropecuária',
'0312404':'Agropecuária',
'0321301':'Agropecuária',
'0321302':'Agropecuária',
'0321303':'Agropecuária',
'0321304':'Agropecuária',
'0321305':'Agropecuária',
'0321399':'Agropecuária',
'0322101':'Agropecuária',
'0322102':'Agropecuária',
'0322103':'Agropecuária',
'0322104':'Agropecuária',
'0322105':'Agropecuária',
'0322106':'Agropecuária',
'0322107':'Agropecuária',
'0322199':'Agropecuária',
'0500302':'Indústria',
'0600001':'Indústria',
'0600002':'Indústria',
'0600003':'Indústria',
'0710301':'Indústria',
'1220403':'Indústria',
'0710302':'Indústria',
'0721901':'Indústria',
'0721902':'Indústria',
'0722701':'Indústria',
'0722702':'Indústria',
'0723501':'Indústria',
'0723502':'Indústria',
'0724301':'Indústria',
'0724302':'Indústria',
'0725100':'Indústria',
'0729401':'Indústria',
'0729402':'Indústria',
'0729403':'Indústria',
'0729404':'Indústria',
'0729405':'Indústria',
'0810001':'Indústria',
'0810002':'Indústria',
'0810003':'Indústria',
'0810004':'Indústria',
'0810005':'Indústria',
'0810006':'Indústria',
'0810007':'Indústria',
'0810008':'Indústria',
'0810009':'Indústria',
'0810010':'Indústria',
'0810099':'Indústria',
'0891600':'Indústria',
'0892402':'Indústria',
'0892403':'Indústria',
'0893200':'Indústria',
'0899101':'Indústria',
'0899102':'Indústria',
'0899103':'Indústria',
'0899199':'Indústria',
'0910600':'Indústria',
'0990401':'Indústria',
'0990402':'Indústria',
'0990403':'Indústria',
'1011201':'Indústria',
'1011202':'Indústria',
'1011203':'Indústria',
'1011204':'Indústria',
'1011205':'Indústria',
'1012101':'Indústria',
'1012102':'Indústria',
'1012103':'Indústria',
'1012104':'Indústria',
'1013901':'Indústria',
'1013902':'Indústria',
'1020101':'Indústria',
'1020102':'Indústria',
'1031700':'Indústria',
'1032501':'Indústria',
'1072401':'Indústria',
'1032599':'Indústria',
'1033301':'Indústria',
'1033302':'Indústria',
'1041400':'Indústria',
'1042200':'Indústria',
'1043100':'Indústria',
'1053800':'Indústria',
'1061901':'Indústria',
'1061902':'Indústria',
'1062700':'Indústria',
'1063500':'Indústria',
'1064300':'Indústria',
'1065101':'Indústria',
'1065102':'Indústria',
'1065103':'Indústria',
'1066000':'Indústria',
'1069400':'Indústria',
'1071600':'Indústria',
'1072402':'Indústria',
'1081301':'Indústria',
'1081302':'Indústria',
'1082100':'Indústria',
'1091100':'Indústria',
'1091101':'Indústria',
'1091102':'Indústria',
'1092900':'Indústria',
'1093701':'Indústria',
'1093702':'Indústria',
'1094500':'Indústria',
'1095300':'Indústria',
'1096100':'Indústria',
'1099601':'Indústria',
'2013400':'Indústria',
'1099602':'Indústria',
'1099603':'Indústria',
'1099604':'Indústria',
'1099605':'Indústria',
'1099606':'Indústria',
'1099607':'Indústria',
'1322700':'Indústria',
'1099699':'Indústria',
'1111901':'Indústria',
'1111902':'Indústria',
'1112700':'Indústria',
'1113501':'Indústria',
'1113502':'Indústria',
'1121600':'Indústria',
'1122401':'Indústria',
'1122402':'Indústria',
'1122403':'Indústria',
'1122404':'Indústria',
'1122499':'Indústria',
'1220401':'Indústria',
'1220402':'Indústria',
'2442300':'Indústria',
'1220499':'Indústria',
'1311100':'Indústria',
'1312000':'Indústria',
'1313800':'Indústria',
'1314600':'Indústria',
'1321900':'Indústria',
'1323500':'Indústria',
'1330800':'Indústria',
'1340501':'Indústria',
'1340502':'Indústria',
'1340599':'Indústria',
'1351100':'Indústria',
'1352900':'Indústria',
'1353700':'Indústria',
'1354500':'Indústria',
'1359600':'Indústria',
'1411801':'Indústria',
'1411802':'Indústria',
'1412601':'Indústria',
'1412602':'Indústria',
'1412603':'Indústria',
'2013401':'Indústria',
'1413401':'Indústria',
'1413402':'Indústria',
'1413403':'Indústria',
'1414200':'Indústria',
'1421500':'Indústria',
'1422300':'Indústria',
'1510600':'Indústria',
'1521100':'Indústria',
'1529700':'Indústria',
'1531901':'Indústria',
'1531902':'Indústria',
'1532700':'Indústria',
'1533500':'Indústria',
'1539400':'Indústria',
'1540800':'Indústria',
'1610201':'Indústria',
'1610202':'Indústria',
'1610203':'Indústria',
'1610204':'Indústria',
'1610205':'Indústria',
'1621800':'Indústria',
'1622601':'Indústria',
'1622602':'Indústria',
'1811302':'Indústria',
'1622699':'Indústria',
'1623400':'Indústria',
'1629301':'Indústria',
'1629302':'Indústria',
'1710900':'Indústria',
'1721400':'Indústria',
'1722200':'Indústria',
'1731100':'Indústria',
'1732000':'Indústria',
'1733800':'Indústria',
'1741901':'Indústria',
'1741902':'Indústria',
'1742701':'Indústria',
'1742702':'Indústria',
'1742799':'Indústria',
'1749400':'Indústria',
'1811301':'Indústria',
'1812100':'Indústria',
'1813001':'Indústria',
'1813099':'Indústria',
'1821100':'Indústria',
'1822900':'Indústria',
'1822901':'Indústria',
'1822999':'Indústria',
'1830001':'Indústria',
'1830002':'Indústria',
'1830003':'Indústria',
'1910100':'Indústria',
'1921700':'Indústria',
'1922501':'Indústria',
'1922502':'Indústria',
'1922599':'Indústria',
'1931400':'Indústria',
'1932200':'Indústria',
'2011800':'Indústria',
'2012600':'Indústria',
'2013402':'Indústria',
'2014200':'Indústria',
'2019301':'Indústria',
'2019399':'Indústria',
'2021500':'Indústria',
'2022300':'Indústria',
'2029100':'Indústria',
'2031200':'Indústria',
'2032100':'Indústria',
'2033900':'Indústria',
'2040100':'Indústria',
'2051700':'Indústria',
'2052500':'Indústria',
'2061400':'Indústria',
'2062200':'Indústria',
'2063100':'Indústria',
'2071100':'Indústria',
'2072000':'Indústria',
'2073800':'Indústria',
'2091600':'Indústria',
'2092401':'Indústria',
'2092402':'Indústria',
'2092403':'Indústria',
'2093200':'Indústria',
'2094100':'Indústria',
'2099101':'Indústria',
'2099199':'Indústria',
'2110600':'Indústria',
'2121101':'Indústria',
'2121102':'Indústria',
'2121103':'Indústria',
'2122000':'Indústria',
'2123800':'Indústria',
'2211100':'Indústria',
'2212900':'Indústria',
'2219600':'Indústria',
'2221800':'Indústria',
'2222600':'Indústria',
'2223400':'Indústria',
'2229301':'Indústria',
'2229302':'Indústria',
'2451200':'Indústria',
'2229303':'Indústria',
'2229399':'Indústria',
'2311700':'Indústria',
'2312500':'Indústria',
'2319200':'Indústria',
'2320600':'Indústria',
'2330301':'Indústria',
'2330302':'Indústria',
'2330303':'Indústria',
'2330304':'Indústria',
'2330305':'Indústria',
'2330399':'Indústria',
'2341900':'Indústria',
'2342701':'Indústria',
'2342702':'Indústria',
'2441502':'Indústria',
'2349401':'Indústria',
'2349499':'Indústria',
'2391501':'Indústria',
'2391502':'Indústria',
'2391503':'Indústria',
'2392300':'Indústria',
'2399101':'Indústria',
'2399102':'Indústria',
'2399199':'Indústria',
'2411300':'Indústria',
'2412100':'Indústria',
'2421100':'Indústria',
'2422901':'Indústria',
'2422902':'Indústria',
'2423701':'Indústria',
'2423702':'Indústria',
'2424501':'Indústria',
'2424502':'Indústria',
'2431800':'Indústria',
'2439300':'Indústria',
'2441501':'Indústria',
'2443100':'Indústria',
'2449101':'Indústria',
'2449102':'Indústria',
'2449103':'Indústria',
'2449199':'Indústria',
'2452100':'Indústria',
'2511000':'Indústria',
'2512800':'Indústria',
'2513600':'Indústria',
'2521700':'Indústria',
'2522500':'Indústria',
'2531401':'Indústria',
'2531402':'Indústria',
'2532201':'Indústria',
'2539000':'Indústria',
'2539001':'Indústria',
'2539002':'Indústria',
'2541100':'Indústria',
'2542000':'Indústria',
'2543800':'Indústria',
'2550101':'Indústria',
'2550102':'Indústria',
'2591800':'Indústria',
'2592601':'Indústria',
'2592602':'Indústria',
'2593400':'Indústria',
'2599301':'Indústria',
'2599302':'Indústria',
'2599399':'Indústria',
'2610800':'Indústria',
'2621300':'Indústria',
'2622100':'Indústria',
'2631100':'Indústria',
'2632900':'Indústria',
'2740601':'Indústria',
'2640000':'Indústria',
'2651500':'Indústria',
'2652300':'Indústria',
'2660400':'Indústria',
'2670101':'Indústria',
'2670102':'Indústria',
'2680900':'Indústria',
'2710401':'Indústria',
'2710402':'Indústria',
'2710403':'Indústria',
'2721000':'Indústria',
'2722801':'Indústria',
'2722802':'Indústria',
'2731700':'Indústria',
'2732500':'Indústria',
'2733300':'Indústria',
'2740602':'Indústria',
'2751100':'Indústria',
'2759701':'Indústria',
'2759799':'Indústria',
'2790201':'Indústria',
'2790202':'Indústria',
'2790299':'Indústria',
'2811900':'Indústria',
'2812700':'Indústria',
'2813500':'Indústria',
'2814302':'Indústria',
'2815101':'Indústria',
'2815102':'Indústria',
'2821601':'Indústria',
'2821602':'Indústria',
'3811400':'Indústria',
'2822401':'Indústria',
'2822402':'Indústria',
'2823200':'Indústria',
'2824101':'Indústria',
'2824102':'Indústria',
'2825900':'Indústria',
'2829101':'Indústria',
'2829199':'Indústria',
'2831300':'Indústria',
'2832100':'Indústria',
'2833000':'Indústria',
'2840200':'Indústria',
'2851800':'Indústria',
'2852600':'Indústria',
'2853400':'Indústria',
'2854200':'Indústria',
'2861500':'Indústria',
'2862300':'Indústria',
'2863100':'Indústria',
'2864000':'Indústria',
'2865800':'Indústria',
'2866600':'Indústria',
'2869100':'Indústria',
'2910701':'Indústria',
'2910702':'Indústria',
'2910703':'Indústria',
'2920402':'Indústria',
'2930101':'Indústria',
'2930102':'Indústria',
'2930103':'Indústria',
'2941700':'Indústria',
'3091102':'Indústria',
'2942500':'Indústria',
'2943300':'Indústria',
'2944100':'Indústria',
'2945000':'Indústria',
'2949201':'Indústria',
'2949299':'Indústria',
'2950600':'Indústria',
'3011301':'Indústria',
'3011302':'Indústria',
'3012100':'Indústria',
'3031800':'Indústria',
'3032600':'Indústria',
'3041500':'Indústria',
'3042300':'Indústria',
'3050400':'Indústria',
'3091100':'Indústria',
'3091101':'Indústria',
'3092000':'Indústria',
'3099700':'Indústria',
'3101200':'Indústria',
'3103900':'Serviços',
'3104700':'Indústria',
'3211601':'Indústria',
'3211602':'Indústria',
'3211603':'Indústria',
'3212400':'Indústria',
'3220500':'Indústria',
'3230200':'Indústria',
'3240001':'Indústria',
'3240002':'Indústria',
'3240003':'Indústria',
'3240099':'Indústria',
'3250701':'Indústria',
'3250702':'Indústria',
'3250703':'Indústria',
'3250704':'Indústria',
'3250705':'Indústria',
'3250706':'Indústria',
'3250707':'Indústria',
'3250708':'Indústria',
'3250709':'Indústria',
'3291400':'Indústria',
'3292201':'Indústria',
'3292202':'Indústria',
'3299001':'Indústria',
'3299002':'Indústria',
'3299003':'Indústria',
'3299004':'Indústria',
'3299005':'Indústria',
'3299006':'Indústria',
'3299099':'Indústria',
'3311200':'Indústria',
'3312101':'Indústria',
'3312102':'Indústria',
'3312103':'Indústria',
'3312104':'Indústria',
'3313901':'Indústria',
'3313902':'Indústria',
'3313999':'Serviços',
'3314701':'Indústria',
'3314702':'Indústria',
'3314703':'Indústria',
'3314704':'Indústria',
'3314705':'Indústria',
'3314706':'Indústria',
'3314707':'Indústria',
'3314708':'Indústria',
'3314709':'Serviços',
'3314710':'Indústria',
'3314711':'Indústria',
'3314712':'Indústria',
'3314713':'Indústria',
'3314714':'Indústria',
'3316302':'Serviços',
'3314715':'Indústria',
'3314716':'Indústria',
'3314717':'Indústria',
'3314718':'Indústria',
'3314719':'Indústria',
'3314720':'Indústria',
'3314721':'Indústria',
'3314722':'Indústria',
'3314799':'Indústria',
'3315500':'Indústria',
'3316301':'Indústria',
'4950700':'Serviços',
'3317101':'Indústria',
'3319800':'Indústria',
'3321000':'Indústria',
'3329501':'Indústria',
'3317102':'Indústria',
'3329599':'Indústria',
'3511500':'Indústria',
'3511501':'Indústria',
'3511502':'Indústria',
'3512300':'Indústria',
'3513100':'Indústria',
'3514000':'Indústria',
'3520401':'Indústria',
'3520402':'Indústria',
'3530100':'Indústria',
'3600601':'Indústria',
'3600602':'Comércio',
'3701100':'Indústria',
'3702900':'Indústria',
'3812200':'Indústria',
'3821100':'Indústria',
'3822000':'Indústria',
'3831999':'Indústria',
'3831901':'Indústria',
'3832700':'Indústria',
'3839401':'Indústria',
'3839499':'Indústria',
'3900500':'Indústria',
'4110700':'Serviços',
'4120400':'Construção Civil',
'4211101':'Construção Civil',
'4211102':'Construção Civil',
'4212000':'Construção Civil',
'4213800':'Construção Civil',
'4221901':'Construção Civil',
'4221902':'Construção Civil',
'4221903':'Construção Civil',
'4221904':'Construção Civil',
'4221905':'Construção Civil',
'4222701':'Construção Civil',
'4223500':'Construção Civil',
'4291000':'Construção Civil',
'4222702':'Construção Civil',
'4292801':'Construção Civil',
'4292802':'Construção Civil',
'4299501':'Construção Civil',
'4299599':'Construção Civil',
'4311801':'Construção Civil',
'4311802':'Construção Civil',
'4312600':'Construção Civil',
'4313400':'Construção Civil',
'4319300':'Construção Civil',
'4321500':'Construção Civil',
'4322301':'Construção Civil',
'4322302':'Construção Civil',
'4322303':'Construção Civil',
'4329101':'Construção Civil',
'4329102':'Construção Civil',
'4329103':'Construção Civil',
'4329104':'Construção Civil',
'4329105':'Construção Civil',
'4329199':'Construção Civil',
'4330401':'Construção Civil',
'4330402':'Construção Civil',
'4330403':'Construção Civil',
'4330404':'Construção Civil',
'4330405':'Construção Civil',
'4330499':'Construção Civil',
'4391600':'Construção Civil',
'4399101':'Construção Civil',
'4399102':'Construção Civil',
'4399103':'Construção Civil',
'4399104':'Construção Civil',
'4399105':'Construção Civil',
'4399199':'Construção Civil',
'4511101':'Comércio',
'4511102':'Comércio',
'4511103':'Comércio',
'8599601':'Serviços',
'4511104':'Comércio',
'4511105':'Comércio',
'4511106':'Comércio',
'4512901':'Comércio',
'4512902':'Comércio',
'4713003':'Comércio',
'4520001':'Comércio',
'4520002':'Comércio',
'4520003':'Comércio',
'4520004':'Comércio',
'4520005':'Comércio',
'4520006':'Comércio',
'4520007':'Comércio',
'4520008':'Comércio',
'4530701':'Comércio',
'4530702':'Comércio',
'4530703':'Comércio',
'4530704':'Comércio',
'4530705':'Comércio',
'4530706':'Comércio',
'4541201':'Comércio',
'4541202':'Comércio',
'4541203':'Comércio',
'4541204':'Comércio',
'4541205':'Comércio',
'4541206':'Comércio',
'4541207':'Comércio',
'4542101':'Comércio',
'4542102':'Comércio',
'4543900':'Comércio',
'4611700':'Comércio',
'4612500':'Comércio',
'4613300':'Comércio',
'4614100':'Comércio',
'5011401':'Serviços',
'4615000':'Comércio',
'4616800':'Comércio',
'4617600':'Comércio',
'4621400':'Comércio',
'4618401':'Comércio',
'4632001':'Comércio',
'4618402':'Comércio',
'4618403':'Comércio',
'4618499':'Comércio',
'4619200':'Comércio',
'4622200':'Comércio',
'4623101':'Comércio',
'4623102':'Comércio',
'4623103':'Comércio',
'4623104':'Comércio',
'4623105':'Comércio',
'4623106':'Comércio',
'4623107':'Comércio',
'4623108':'Comércio',
'4623109':'Comércio',
'4623199':'Comércio',
'4631100':'Comércio',
'4632002':'Comércio',
'4632003':'Comércio',
'4633801':'Comércio',
'4633802':'Comércio',
'4633803':'Comércio',
'4634601':'Comércio',
'4634602':'Comércio',
'4634603':'Comércio',
'4634699':'Comércio',
'4635401':'Comércio',
'4635402':'Comércio',
'4635403':'Comércio',
'4635499':'Comércio',
'8599602':'Serviços',
'4636201':'Comércio',
'4636202':'Comércio',
'4637101':'Comércio',
'4637102':'Comércio',
'4782201':'Comércio',
'4637103':'Comércio',
'4637104':'Comércio',
'4637105':'Comércio',
'4637106':'Comércio',
'4637107':'Comércio',
'4637199':'Comércio',
'4639701':'Comércio',
'4639702':'Comércio',
'4641901':'Comércio',
'4641902':'Comércio',
'4641903':'Comércio',
'4642701':'Comércio',
'4642702':'Comércio',
'4643501':'Comércio',
'4643502':'Comércio',
'4644301':'Comércio',
'4644302':'Comércio',
'4645101':'Comércio',
'4645102':'Comércio',
'4645103':'Comércio',
'4646001':'Comércio',
'4646002':'Comércio',
'4647801':'Comércio',
'4647802':'Comércio',
'5011402':'Serviços',
'4649401':'Comércio',
'4649402':'Comércio',
'4649403':'Comércio',
'4649404':'Comércio',
'4649405':'Comércio',
'4649406':'Comércio',
'4649407':'Comércio',
'4649408':'Comércio',
'4673700':'Comércio',
'4649409':'Comércio',
'4649410':'Comércio',
'4649499':'Comércio',
'4651601':'Comércio',
'4651602':'Comércio',
'4652400':'Comércio',
'4661300':'Comércio',
'4662100':'Comércio',
'4663000':'Comércio',
'4664800':'Comércio',
'4665600':'Comércio',
'4669901':'Comércio',
'4669999':'Comércio',
'4671100':'Comércio',
'4672900':'Comércio',
'4674500':'Comércio',
'4679601':'Comércio',
'4679602':'Comércio',
'4679603':'Comércio',
'4679604':'Comércio',
'4679699':'Comércio',
'4681801':'Comércio',
'4681802':'Comércio',
'4681803':'Comércio',
'4681804':'Comércio',
'4681805':'Comércio',
'4682600':'Comércio',
'4683400':'Comércio',
'4684201':'Comércio',
'4684202':'Comércio',
'4684299':'Comércio',
'4685100':'Comércio',
'4686901':'Comércio',
'4686902':'Comércio',
'4687701':'Comércio',
'4687702':'Comércio',
'4687703':'Comércio',
'4689301':'Comércio',
'4689302':'Comércio',
'4689399':'Comércio',
'4691500':'Comércio',
'4692300':'Comércio',
'4693100':'Comércio',
'4711301':'Comércio',
'4711302':'Comércio',
'4712100':'Comércio',
'4713001':'Comércio',
'4713002':'Comércio',
'6141800':'Serviços',
'4713004':'Comércio',
'4713005':'Comércio',
'4721101':'Comércio',
'4721102':'Comércio',
'4721103':'Comércio',
'4721104':'Comércio',
'4722901':'Comércio',
'4722902':'Comércio',
'4724500':'Comércio',
'4729601':'Comércio',
'4729602':'Comércio',
'4729699':'Comércio',
'4731800':'Comércio',
'4732600':'Comércio',
'4741500':'Comércio',
'4742300':'Comércio',
'4743100':'Comércio',
'4744001':'Comércio',
'4744002':'Comércio',
'4744003':'Comércio',
'4744004':'Comércio',
'4744005':'Comércio',
'4744006':'Comércio',
'4744099':'Comércio',
'4751200':'Comércio',
'4751201':'Comércio',
'4751202':'Comércio',
'4752100':'Comércio',
'4753900':'Comércio',
'4754701':'Comércio',
'4754702':'Comércio',
'4754703':'Comércio',
'4755501':'Comércio',
'4755502':'Comércio',
'4755503':'Comércio',
'4756300':'Comércio',
'4757100':'Comércio',
'4781400':'Comércio',
'4759801':'Comércio',
'5012201':'Serviços',
'4759899':'Comércio',
'4761001':'Comércio',
'4761002':'Comércio',
'4761003':'Comércio',
'4762800':'Comércio',
'4763601':'Comércio',
'4763602':'Comércio',
'4763603':'Comércio',
'4763604':'Comércio',
'4763605':'Comércio',
'4771701':'Comércio',
'4771702':'Comércio',
'4771703':'Comércio',
'4771704':'Comércio',
'4772500':'Comércio',
'4773300':'Comércio',
'4774100':'Comércio',
'4782202':'Comércio',
'4783101':'Comércio',
'4783102':'Comércio',
'4784900':'Comércio',
'4785701':'Comércio',
'4785799':'Comércio',
'4789001':'Comércio',
'4789002':'Comércio',
'4789003':'Comércio',
'4789004':'Comércio',
'4789005':'Comércio',
'4789006':'Comércio',
'4789007':'Comércio',
'4789008':'Comércio',
'4789009':'Comércio',
'4789099':'Comércio',
'4912401':'Serviços',
'4912402':'Serviços',
'4912403':'Serviços',
'4921301':'Serviços',
'4921302':'Serviços',
'4922101':'Serviços',
'4922102':'Serviços',
'4922103':'Serviços',
'4923001':'Serviços',
'4923002':'Serviços',
'4924800':'Serviços',
'4929901':'Serviços',
'4929902':'Serviços',
'4929903':'Serviços',
'4929904':'Serviços',
'4929999':'Serviços',
'4930201':'Serviços',
'4930202':'Serviços',
'4930204':'Serviços',
'5012202':'Serviços',
'5021101':'Serviços',
'5021102':'Serviços',
'5022001':'Serviços',
'5022002':'Serviços',
'5030101':'Serviços',
'5030102':'Serviços',
'5030103':'Serviços',
'5091201':'Serviços',
'5091202':'Serviços',
'5099801':'Serviços',
'5099899':'Serviços',
'5111100':'Serviços',
'5130700':'Serviços',
'5112901':'Serviços',
'5112999':'Serviços',
'5120000':'Serviços',
'5211701':'Serviços',
'5211702':'Serviços',
'5211799':'Serviços',
'5212500':'Serviços',
'5221400':'Serviços',
'5222200':'Serviços',
'5223100':'Serviços',
'5229001':'Serviços',
'5229002':'Comércio',
'5229099':'Serviços',
'5231101':'Serviços',
'5231102':'Serviços',
'5231103':'Serviços',
'5232000':'Serviços',
'5239700':'Serviços',
'5239701':'Serviços',
'5239799':'Serviços',
'5240101':'Serviços',
'5310501':'Serviços',
'5240199':'Serviços',
'5250801':'Serviços',
'5250802':'Serviços',
'5250803':'Serviços',
'5250804':'Serviços',
'5250805':'Serviços',
'5310502':'Serviços',
'5320201':'Serviços',
'5320202':'Serviços',
'5510801':'Serviços',
'5510802':'Serviços',
'5510803':'Serviços',
'5590601':'Serviços',
'5590602':'Serviços',
'5590603':'Serviços',
'5590699':'Serviços',
'5611201':'Serviços',
'7311400':'Serviços',
'5611202':'Serviços',
'5611203':'Serviços',
'5611204':'Serviços',
'5611205':'Serviços',
'5620101':'Indústria',
'5620102':'Indústria',
'5620103':'Serviços',
'5620104':'Indústria',
'5811500':'Indústria',
'5812300':'Indústria',
'5812301':'Indústria',
'5812302':'Indústria',
'5813100':'Indústria',
'5819100':'Indústria',
'5821200':'Indústria',
'5822100':'Indústria',
'5822101':'Indústria',
'5822102':'Indústria',
'5823900':'Indústria',
'5829800':'Indústria',
'5911101':'Indústria',
'5911102':'Indústria',
'5911199':'Indústria',
'5912001':'Indústria',
'5912002':'Indústria',
'5912099':'Indústria',
'5913800':'Serviços',
'5914600':'Serviços',
'6203100':'Serviços',
'5920100':'Indústria',
'6021700':'Serviços',
'6022501':'Serviços',
'6022502':'Serviços',
'6110801':'Serviços',
'6110802':'Serviços',
'6110803':'Serviços',
'6110899':'Serviços',
'6120501':'Serviços',
'6120502':'Serviços',
'6120599':'Serviços',
'6130200':'Serviços',
'6142600':'Serviços',
'6143400':'Serviços',
'6190601':'Serviços',
'6190602':'Serviços',
'6190699':'Serviços',
'6201500':'Serviços',
'6201501':'Serviços',
'6201502':'Serviços',
'6202300':'Serviços',
'6911702':'Serviços',
'6204000':'Serviços',
'6209100':'Serviços',
'6311900':'Serviços',
'6319400':'Serviços',
'6391700':'Serviços',
'6399200':'Serviços',
'6410700':'Serviços',
'6421200':'Serviços',
'6422100':'Serviços',
'6423900':'Serviços',
'6424701':'Serviços',
'6424702':'Serviços',
'8599604':'Serviços',
'6424703':'Serviços',
'6424704':'Serviços',
'6431000':'Serviços',
'6432800':'Serviços',
'6433600':'Serviços',
'6434400':'Serviços',
'6435201':'Serviços',
'6435202':'Serviços',
'6435203':'Serviços',
'8593700':'Serviços',
'6436100':'Serviços',
'6437900':'Serviços',
'6438701':'Serviços',
'6438799':'Serviços',
'6440900':'Serviços',
'6450600':'Serviços',
'6461100':'Serviços',
'6462000':'Serviços',
'6463800':'Serviços',
'6470101':'Serviços',
'6470102':'Serviços',
'6470103':'Serviços',
'6491300':'Serviços',
'6492100':'Serviços',
'6499901':'Serviços',
'6499902':'Serviços',
'6499903':'Serviços',
'6499904':'Serviços',
'6499905':'Serviços',
'6499999':'Serviços',
'6511101':'Serviços',
'6511102':'Serviços',
'6512000':'Serviços',
'6520100':'Serviços',
'6530800':'Serviços',
'6541300':'Serviços',
'6542100':'Serviços',
'6550200':'Serviços',
'6611801':'Serviços',
'6611802':'Serviços',
'6611803':'Serviços',
'6611804':'Serviços',
'6612601':'Serviços',
'6612602':'Serviços',
'6612603':'Serviços',
'6612604':'Serviços',
'6612605':'Serviços',
'6613400':'Serviços',
'6619301':'Serviços',
'6619302':'Serviços',
'6619303':'Serviços',
'6619304':'Serviços',
'6619305':'Serviços',
'6619399':'Serviços',
'6621501':'Serviços',
'6621502':'Serviços',
'6622300':'Serviços',
'6629100':'Serviços',
'6630400':'Serviços',
'6810201':'Serviços',
'6810202':'Serviços',
'6810203':'Serviços',
'6821801':'Serviços',
'6821802':'Serviços',
'6822600':'Serviços',
'6911701':'Serviços',
'6911703':'Serviços',
'6912500':'Serviços',
'6920601':'Serviços',
'6920602':'Serviços',
'7020400':'Serviços',
'7111100':'Serviços',
'7112000':'Serviços',
'7119701':'Serviços',
'7119702':'Serviços',
'7119703':'Serviços',
'7119704':'Serviços',
'7119799':'Serviços',
'7120100':'Serviços',
'7210000':'Serviços',
'7220700':'Serviços',
'7312200':'Serviços',
'7319001':'Serviços',
'7319002':'Serviços',
'7319003':'Serviços',
'7319004':'Serviços',
'7319099':'Serviços',
'7320300':'Serviços',
'7410201':'Serviços',
'7410202':'Serviços',
'7410203':'Serviços',
'7410299':'Serviços',
'7420001':'Serviços',
'7420002':'Serviços',
'7420003':'Serviços',
'7420004':'Serviços',
'7420005':'Serviços',
'7490101':'Serviços',
'7490102':'Serviços',
'7490103':'Serviços',
'7490104':'Serviços',
'7490105':'Serviços',
'8599605':'Serviços',
'7500100':'Serviços',
'7490199':'Serviços',
'7711000':'Serviços',
'7719501':'Serviços',
'7719502':'Serviços',
'7719599':'Serviços',
'7721700':'Serviços',
'7722500':'Serviços',
'7723300':'Serviços',
'7729201':'Serviços',
'7729202':'Serviços',
'7729203':'Serviços',
'7729299':'Serviços',
'7731400':'Serviços',
'7732201':'Serviços',
'7732202':'Serviços',
'7733100':'Serviços',
'7739001':'Serviços',
'7739002':'Serviços',
'8599699':'Serviços',
'7739003':'Serviços',
'7739099':'Serviços',
'7740300':'Serviços',
'7810800':'Serviços',
'7820500':'Serviços',
'7830200':'Serviços',
'7911200':'Serviços',
'7912100':'Serviços',
'7990200':'Serviços',
'8011101':'Serviços',
'8011102':'Serviços',
'8012900':'Serviços',
'8020000':'Serviços',
'8020001':'Serviços',
'8020002':'Serviços',
'8030700':'Serviços',
'8111700':'Serviços',
'8112500':'Serviços',
'8121400':'Serviços',
'8122200':'Serviços',
'8129000':'Serviços',
'8130300':'Agropecuária',
'8211300':'Serviços',
'8219901':'Serviços',
'8219999':'Serviços',
'8220200':'Serviços',
'8230001':'Serviços',
'8230002':'Serviços',
'8291100':'Serviços',
'8292000':'Serviços',
'8299701':'Serviços',
'8299702':'Serviços',
'8299703':'Serviços',
'8299704':'Serviços',
'8299705':'Serviços',
'8299706':'Serviços',
'8299707':'Serviços',
'8299799':'Serviços',
'8411600':'Administração Pública',
'8412400':'Administração Pública',
'8413200':'Administração Pública',
'8421300':'Administração Pública',
'8422100':'Administração Pública',
'8423000':'Administração Pública',
'8424800':'Administração Pública',
'8425600':'Administração Pública',
'8430200':'Serviços',
'8511200':'Serviços',
'8512100':'Serviços',
'8513900':'Serviços',
'8520100':'Serviços',
'8531700':'Serviços',
'8532500':'Serviços',
'8533300':'Serviços',
'8541400':'Serviços',
'8542200':'Serviços',
'8550301':'Serviços',
'8550302':'Serviços',
'8591100':'Serviços',
'8592901':'Serviços',
'8592902':'Serviços',
'8592903':'Serviços',
'8592999':'Serviços',
'8610101':'Serviços',
'8610102':'Serviços',
'8621601':'Serviços',
'8621602':'Serviços',
'8622400':'Serviços',
'8630501':'Serviços',
'8630502':'Serviços',
'8630503':'Serviços',
'8630504':'Serviços',
'8630505':'Serviços',
'8630506':'Serviços',
'8630507':'Serviços',
'8630599':'Serviços',
'8640201':'Serviços',
'8640202':'Serviços',
'8640203':'Serviços',
'8640204':'Serviços',
'8640205':'Serviços',
'8640206':'Serviços',
'8640207':'Serviços',
'8640208':'Serviços',
'8640209':'Serviços',
'8640210':'Serviços',
'8640211':'Serviços',
'8640212':'Serviços',
'8640213':'Serviços',
'8640214':'Serviços',
'8640299':'Serviços',
'8650001':'Serviços',
'8650002':'Serviços',
'8650003':'Serviços',
'8650004':'Serviços',
'8650005':'Serviços',
'8650006':'Serviços',
'9491000':'Serviços',
'8650099':'Serviços',
'8660700':'Serviços',
'8690901':'Serviços',
'8690902':'Serviços',
'8650007':'Serviços',
'8690903':'Serviços',
'8690904':'Serviços',
'8690999':'Serviços',
'8711501':'Serviços',
'8711502':'Serviços',
'8711503':'Serviços',
'8711504':'Serviços',
'8711505':'Serviços',
'8712300':'Serviços',
'8720401':'Serviços',
'8720499':'Serviços',
'8730101':'Serviços',
'8730102':'Serviços',
'8730199':'Serviços',
'8800600':'Serviços',
'9001901':'Serviços',
'9001902':'Serviços',
'9001903':'Serviços',
'9001904':'Serviços',
'9001905':'Serviços',
'9001906':'Serviços',
'9001999':'Serviços',
'9002701':'Serviços',
'9002702':'Serviços',
'9003500':'Serviços',
'9101500':'Serviços',
'9102301':'Serviços',
'9102302':'Serviços',
'9103100':'Serviços',
'9200301':'Serviços',
'9200302':'Serviços',
'9200399':'Serviços',
'9311500':'Serviços',
'9312300':'Serviços',
'9313100':'Serviços',
'9319199':'Serviços',
'9321200':'Serviços',
'9329801':'Serviços',
'9329802':'Serviços',
'9329803':'Serviços',
'9329804':'Serviços',
'9329899':'Serviços',
'9411100':'Serviços',
'9412000':'Serviços',
'9412001':'Serviços',
'9412099':'Serviços',
'9420100':'Serviços',
'9430800':'Serviços',
'9492800':'Serviços',
'9493600':'Serviços',
'9499500':'Serviços',
'9601701':'Serviços',
'9511800':'Serviços',
'9512600':'Comércio',
'9521500':'Comércio',
'9529101':'Comércio',
'9529102':'Comércio',
'9529103':'Comércio',
'9529104':'Comércio',
'9529105':'Comércio',
'9529106':'Comércio',
'9529199':'Comércio',
'9601703':'Serviços',
'9602501':'Serviços',
'9602502':'Serviços',
'9603302':'Serviços',
'9603303':'Serviços',
'9603304':'Serviços',
'9603305':'Serviços',
'0111301':'Agropecuária',
'0111302':'Agropecuária',
'0111303':'Agropecuária',
'0111399':'Agropecuária',
'9603399':'Serviços',
'9609201':'Serviços',
'9609202':'Serviços',
'9609203':'Serviços',
'9609204':'Serviços',
'1051100':'Indústria',
'9319101':'Serviços',
'9601702':'Serviços',
'9609205':'Serviços',
'9609206':'Serviços',
'9609207':'Serviços',
'9609208':'Serviços',
'9609299':'Serviços',
'9900800':'Serviços',
'9999999':'-1',
"9999997" : "-1",
"-1":"-1"
}

# (NOVO) Classificação do IBGE que converte a cnae 2 subclasse para setor de ativ. econômica do ibge. Como a RAIS ESTAB. não tem setor ibge, vamos adicionar para criar o porte sebrae, que usa o setor ibge para sua padronização.    
dpara_cnae2_setoribge = {'0112101':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0500301':'Extrativa Mineral',
'0892401':'Extrativa Mineral',
'1052000':'Indústria de Transformação',
'1210700':'Indústria de Transformação',
'0112102':'Agropecuária, Extração Vegetal, Caça e Pesca',
'2532202':'Indústria de Transformação',
'2814301':'Indústria de Transformação',
'2920401':'Indústria de Transformação',
'3102100':'Indústria de Transformação',
'4940000':'Serviços',
'0112199':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0113000':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0114800':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0115600':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0116401':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0116402':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0116403':'Agropecuária, Extração Vegetal, Caça e Pesca',
'8599603':'Serviços',
'4723700':'Comércio',
'4911600':'Serviços',
'4930203':'Serviços',
'0116499':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0119901':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0119902':'Agropecuária, Extração Vegetal, Caça e Pesca',
'5612100':'Serviços',
'6010100':'Serviços',
'6493000':'Serviços',
'0119903':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0119904':'Agropecuária, Extração Vegetal, Caça e Pesca',
'9603301':'Serviços',
'0119905':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0119906':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0119907':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0119908':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0119909':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0119999':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0121101':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0121102':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0122900':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0131800':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0132600':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0133401':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0133402':'Agropecuária, Extração Vegetal, Caça e Pesca',
'9700500':'Serviços',
'0133403':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0133404':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0133405':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0133406':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0133407':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0133408':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0133409':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0133410':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0133411':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0133499':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0134200':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0135100':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0151201':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0139301':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0139302':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0139303':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0139304':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0139305':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0139306':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0139399':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0141501':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0141502':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0142300':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0151202':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0151203':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0152101':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0152102':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0152103':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0153901':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0153902':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0154700':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0155501':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0155502':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0155503':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0155504':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0155505':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0159801':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0159802':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0159803':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0159804':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0159899':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0161001':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0161002':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0161003':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0161099':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0162801':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0162802':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0162803':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0162899':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0163600':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0170900':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0210101':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0210102':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0210103':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0210104':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0210105':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0210106':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0210107':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0210108':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0210109':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0210199':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0220901':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0220902':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0220903':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0220904':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0220905':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0220906':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0220999':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0230600':'Serviços',
'0311601':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0311602':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0311603':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0311604':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0312401':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0312402':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0312403':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0312404':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0321301':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0321302':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0321303':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0321304':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0321305':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0321399':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0322101':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0322102':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0322103':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0322104':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0322105':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0322106':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0322107':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0322199':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0500302':'Extrativa Mineral',
'0600001':'Extrativa Mineral',
'0600002':'Extrativa Mineral',
'0600003':'Extrativa Mineral',
'0710301':'Extrativa Mineral',
'1220403':'Indústria de Transformação',
'0710302':'Extrativa Mineral',
'0721901':'Extrativa Mineral',
'0721902':'Extrativa Mineral',
'0722701':'Extrativa Mineral',
'0722702':'Extrativa Mineral',
'0723501':'Extrativa Mineral',
'0723502':'Extrativa Mineral',
'0724301':'Extrativa Mineral',
'0724302':'Extrativa Mineral',
'0725100':'Extrativa Mineral',
'0729401':'Extrativa Mineral',
'0729402':'Extrativa Mineral',
'0729403':'Extrativa Mineral',
'0729404':'Extrativa Mineral',
'0729405':'Extrativa Mineral',
'0810001':'Extrativa Mineral',
'0810002':'Extrativa Mineral',
'0810003':'Extrativa Mineral',
'0810004':'Extrativa Mineral',
'0810005':'Extrativa Mineral',
'0810006':'Extrativa Mineral',
'0810007':'Extrativa Mineral',
'0810008':'Extrativa Mineral',
'0810009':'Extrativa Mineral',
'0810010':'Extrativa Mineral',
'0810099':'Extrativa Mineral',
'0891600':'Extrativa Mineral',
'0892402':'Extrativa Mineral',
'0892403':'Extrativa Mineral',
'0893200':'Extrativa Mineral',
'0899101':'Extrativa Mineral',
'0899102':'Extrativa Mineral',
'0899103':'Extrativa Mineral',
'0899199':'Extrativa Mineral',
'0910600':'Extrativa Mineral',
'0990401':'Extrativa Mineral',
'0990402':'Extrativa Mineral',
'0990403':'Extrativa Mineral',
'1011201':'Indústria de Transformação',
'1011202':'Indústria de Transformação',
'1011203':'Indústria de Transformação',
'1011204':'Indústria de Transformação',
'1011205':'Indústria de Transformação',
'1012101':'Indústria de Transformação',
'1012102':'Indústria de Transformação',
'1012103':'Indústria de Transformação',
'1012104':'Indústria de Transformação',
'1013901':'Indústria de Transformação',
'1013902':'Indústria de Transformação',
'1020101':'Indústria de Transformação',
'1020102':'Indústria de Transformação',
'1031700':'Indústria de Transformação',
'1032501':'Indústria de Transformação',
'1072401':'Indústria de Transformação',
'1032599':'Indústria de Transformação',
'1033301':'Indústria de Transformação',
'1033302':'Indústria de Transformação',
'1041400':'Indústria de Transformação',
'1042200':'Indústria de Transformação',
'1043100':'Indústria de Transformação',
'1053800':'Indústria de Transformação',
'1061901':'Indústria de Transformação',
'1061902':'Indústria de Transformação',
'1062700':'Indústria de Transformação',
'1063500':'Indústria de Transformação',
'1064300':'Indústria de Transformação',
'1065101':'Indústria de Transformação',
'1065102':'Indústria de Transformação',
'1065103':'Indústria de Transformação',
'1066000':'Indústria de Transformação',
'1069400':'Indústria de Transformação',
'1071600':'Indústria de Transformação',
'1072402':'Indústria de Transformação',
'1081301':'Indústria de Transformação',
'1081302':'Indústria de Transformação',
'1082100':'Indústria de Transformação',
'1091100':'Indústria de Transformação',
'1091101':'Indústria de Transformação',
'1091102':'Indústria de Transformação',
'1092900':'Indústria de Transformação',
'1093701':'Indústria de Transformação',
'1093702':'Indústria de Transformação',
'1094500':'Indústria de Transformação',
'1095300':'Indústria de Transformação',
'1096100':'Indústria de Transformação',
'1099601':'Indústria de Transformação',
'2013400':'Indústria de Transformação',
'1099602':'Indústria de Transformação',
'1099603':'Indústria de Transformação',
'1099604':'Indústria de Transformação',
'1099605':'Indústria de Transformação',
'1099606':'Indústria de Transformação',
'1099607':'Indústria de Transformação',
'1322700':'Indústria de Transformação',
'1099699':'Indústria de Transformação',
'1111901':'Indústria de Transformação',
'1111902':'Indústria de Transformação',
'1112700':'Indústria de Transformação',
'1113501':'Indústria de Transformação',
'1113502':'Indústria de Transformação',
'1121600':'Indústria de Transformação',
'1122401':'Indústria de Transformação',
'1122402':'Indústria de Transformação',
'1122403':'Indústria de Transformação',
'1122404':'Indústria de Transformação',
'1122499':'Indústria de Transformação',
'1220401':'Indústria de Transformação',
'1220402':'Indústria de Transformação',
'2442300':'Indústria de Transformação',
'1220499':'Indústria de Transformação',
'1311100':'Indústria de Transformação',
'1312000':'Indústria de Transformação',
'1313800':'Indústria de Transformação',
'1314600':'Indústria de Transformação',
'1321900':'Indústria de Transformação',
'1323500':'Indústria de Transformação',
'1330800':'Indústria de Transformação',
'1340501':'Indústria de Transformação',
'1340502':'Indústria de Transformação',
'1340599':'Indústria de Transformação',
'1351100':'Indústria de Transformação',
'1352900':'Indústria de Transformação',
'1353700':'Indústria de Transformação',
'1354500':'Indústria de Transformação',
'1359600':'Indústria de Transformação',
'1411801':'Indústria de Transformação',
'1411802':'Indústria de Transformação',
'1412601':'Indústria de Transformação',
'1412602':'Indústria de Transformação',
'1412603':'Indústria de Transformação',
'2013401':'Indústria de Transformação',
'1413401':'Indústria de Transformação',
'1413402':'Indústria de Transformação',
'1413403':'Indústria de Transformação',
'1414200':'Indústria de Transformação',
'1421500':'Indústria de Transformação',
'1422300':'Indústria de Transformação',
'1510600':'Indústria de Transformação',
'1521100':'Indústria de Transformação',
'1529700':'Indústria de Transformação',
'1531901':'Indústria de Transformação',
'1531902':'Indústria de Transformação',
'1532700':'Indústria de Transformação',
'1533500':'Indústria de Transformação',
'1539400':'Indústria de Transformação',
'1540800':'Indústria de Transformação',
'1610201':'Indústria de Transformação',
'1610202':'Indústria de Transformação',
'1610203':'Indústria de Transformação',
'1610204':'Indústria de Transformação',
'1610205':'Indústria de Transformação',
'1621800':'Indústria de Transformação',
'1622601':'Indústria de Transformação',
'1622602':'Indústria de Transformação',
'1811302':'Indústria de Transformação',
'1622699':'Indústria de Transformação',
'1623400':'Indústria de Transformação',
'1629301':'Indústria de Transformação',
'1629302':'Indústria de Transformação',
'1710900':'Indústria de Transformação',
'1721400':'Indústria de Transformação',
'1722200':'Indústria de Transformação',
'1731100':'Indústria de Transformação',
'1732000':'Indústria de Transformação',
'1733800':'Indústria de Transformação',
'1741901':'Indústria de Transformação',
'1741902':'Indústria de Transformação',
'1742701':'Indústria de Transformação',
'1742702':'Indústria de Transformação',
'1742799':'Indústria de Transformação',
'1749400':'Indústria de Transformação',
'1811301':'Indústria de Transformação',
'1812100':'Indústria de Transformação',
'1813001':'Indústria de Transformação',
'1813099':'Indústria de Transformação',
'1821100':'Indústria de Transformação',
'1822900':'Indústria de Transformação',
'1822901':'Indústria de Transformação',
'1822999':'Indústria de Transformação',
'1830001':'Indústria de Transformação',
'1830002':'Indústria de Transformação',
'1830003':'Indústria de Transformação',
'1910100':'Indústria de Transformação',
'1921700':'Indústria de Transformação',
'1922501':'Indústria de Transformação',
'1922502':'Indústria de Transformação',
'1922599':'Indústria de Transformação',
'1931400':'Indústria de Transformação',
'1932200':'Indústria de Transformação',
'2011800':'Indústria de Transformação',
'2012600':'Indústria de Transformação',
'2013402':'Indústria de Transformação',
'2014200':'Indústria de Transformação',
'2019301':'Indústria de Transformação',
'2019399':'Indústria de Transformação',
'2021500':'Indústria de Transformação',
'2022300':'Indústria de Transformação',
'2029100':'Indústria de Transformação',
'2031200':'Indústria de Transformação',
'2032100':'Indústria de Transformação',
'2033900':'Indústria de Transformação',
'2040100':'Indústria de Transformação',
'2051700':'Indústria de Transformação',
'2052500':'Indústria de Transformação',
'2061400':'Indústria de Transformação',
'2062200':'Indústria de Transformação',
'2063100':'Indústria de Transformação',
'2071100':'Indústria de Transformação',
'2072000':'Indústria de Transformação',
'2073800':'Indústria de Transformação',
'2091600':'Indústria de Transformação',
'2092401':'Indústria de Transformação',
'2092402':'Indústria de Transformação',
'2092403':'Indústria de Transformação',
'2093200':'Indústria de Transformação',
'2094100':'Indústria de Transformação',
'2099101':'Indústria de Transformação',
'2099199':'Indústria de Transformação',
'2110600':'Indústria de Transformação',
'2121101':'Indústria de Transformação',
'2121102':'Indústria de Transformação',
'2121103':'Indústria de Transformação',
'2122000':'Indústria de Transformação',
'2123800':'Indústria de Transformação',
'2211100':'Indústria de Transformação',
'2212900':'Indústria de Transformação',
'2219600':'Indústria de Transformação',
'2221800':'Indústria de Transformação',
'2222600':'Indústria de Transformação',
'2223400':'Indústria de Transformação',
'2229301':'Indústria de Transformação',
'2229302':'Indústria de Transformação',
'2451200':'Indústria de Transformação',
'2229303':'Indústria de Transformação',
'2229399':'Indústria de Transformação',
'2311700':'Indústria de Transformação',
'2312500':'Indústria de Transformação',
'2319200':'Indústria de Transformação',
'2320600':'Indústria de Transformação',
'2330301':'Indústria de Transformação',
'2330302':'Indústria de Transformação',
'2330303':'Indústria de Transformação',
'2330304':'Indústria de Transformação',
'2330305':'Indústria de Transformação',
'2330399':'Indústria de Transformação',
'2341900':'Indústria de Transformação',
'2342701':'Indústria de Transformação',
'2342702':'Indústria de Transformação',
'2441502':'Indústria de Transformação',
'2349401':'Indústria de Transformação',
'2349499':'Indústria de Transformação',
'2391501':'Indústria de Transformação',
'2391502':'Indústria de Transformação',
'2391503':'Indústria de Transformação',
'2392300':'Indústria de Transformação',
'2399101':'Indústria de Transformação',
'2399102':'Indústria de Transformação',
'2399199':'Indústria de Transformação',
'2411300':'Indústria de Transformação',
'2412100':'Indústria de Transformação',
'2421100':'Indústria de Transformação',
'2422901':'Indústria de Transformação',
'2422902':'Indústria de Transformação',
'2423701':'Indústria de Transformação',
'2423702':'Indústria de Transformação',
'2424501':'Indústria de Transformação',
'2424502':'Indústria de Transformação',
'2431800':'Indústria de Transformação',
'2439300':'Indústria de Transformação',
'2441501':'Indústria de Transformação',
'2443100':'Indústria de Transformação',
'2449101':'Indústria de Transformação',
'2449102':'Indústria de Transformação',
'2449103':'Indústria de Transformação',
'2449199':'Indústria de Transformação',
'2452100':'Indústria de Transformação',
'2511000':'Indústria de Transformação',
'2512800':'Indústria de Transformação',
'2513600':'Indústria de Transformação',
'2521700':'Indústria de Transformação',
'2522500':'Indústria de Transformação',
'2531401':'Indústria de Transformação',
'2531402':'Indústria de Transformação',
'2532201':'Indústria de Transformação',
'2539000':'Indústria de Transformação',
'2539001':'Indústria de Transformação',
'2539002':'Indústria de Transformação',
'2541100':'Indústria de Transformação',
'2542000':'Indústria de Transformação',
'2543800':'Indústria de Transformação',
'2550101':'Indústria de Transformação',
'2550102':'Indústria de Transformação',
'2591800':'Indústria de Transformação',
'2592601':'Indústria de Transformação',
'2592602':'Indústria de Transformação',
'2593400':'Indústria de Transformação',
'2599301':'Indústria de Transformação',
'2599302':'Indústria de Transformação',
'2599399':'Indústria de Transformação',
'2610800':'Indústria de Transformação',
'2621300':'Indústria de Transformação',
'2622100':'Indústria de Transformação',
'2631100':'Indústria de Transformação',
'2632900':'Indústria de Transformação',
'2740601':'Indústria de Transformação',
'2640000':'Indústria de Transformação',
'2651500':'Indústria de Transformação',
'2652300':'Indústria de Transformação',
'2660400':'Indústria de Transformação',
'2670101':'Indústria de Transformação',
'2670102':'Indústria de Transformação',
'2680900':'Indústria de Transformação',
'2710401':'Indústria de Transformação',
'2710402':'Indústria de Transformação',
'2710403':'Indústria de Transformação',
'2721000':'Indústria de Transformação',
'2722801':'Indústria de Transformação',
'2722802':'Indústria de Transformação',
'2731700':'Indústria de Transformação',
'2732500':'Indústria de Transformação',
'2733300':'Indústria de Transformação',
'2740602':'Indústria de Transformação',
'2751100':'Indústria de Transformação',
'2759701':'Indústria de Transformação',
'2759799':'Indústria de Transformação',
'2790201':'Indústria de Transformação',
'2790202':'Indústria de Transformação',
'2790299':'Indústria de Transformação',
'2811900':'Indústria de Transformação',
'2812700':'Indústria de Transformação',
'2813500':'Indústria de Transformação',
'2814302':'Indústria de Transformação',
'2815101':'Indústria de Transformação',
'2815102':'Indústria de Transformação',
'2821601':'Indústria de Transformação',
'2821602':'Indústria de Transformação',
'3811400':'Serviços Industriais de Utilidade Pública',
'2822401':'Indústria de Transformação',
'2822402':'Indústria de Transformação',
'2823200':'Indústria de Transformação',
'2824101':'Indústria de Transformação',
'2824102':'Indústria de Transformação',
'2825900':'Indústria de Transformação',
'2829101':'Indústria de Transformação',
'2829199':'Indústria de Transformação',
'2831300':'Indústria de Transformação',
'2832100':'Indústria de Transformação',
'2833000':'Indústria de Transformação',
'2840200':'Indústria de Transformação',
'2851800':'Indústria de Transformação',
'2852600':'Indústria de Transformação',
'2853400':'Indústria de Transformação',
'2854200':'Indústria de Transformação',
'2861500':'Indústria de Transformação',
'2862300':'Indústria de Transformação',
'2863100':'Indústria de Transformação',
'2864000':'Indústria de Transformação',
'2865800':'Indústria de Transformação',
'2866600':'Indústria de Transformação',
'2869100':'Indústria de Transformação',
'2910701':'Indústria de Transformação',
'2910702':'Indústria de Transformação',
'2910703':'Indústria de Transformação',
'2920402':'Indústria de Transformação',
'2930101':'Indústria de Transformação',
'2930102':'Indústria de Transformação',
'2930103':'Indústria de Transformação',
'2941700':'Indústria de Transformação',
'3091102':'Indústria de Transformação',
'2942500':'Indústria de Transformação',
'2943300':'Indústria de Transformação',
'2944100':'Indústria de Transformação',
'2945000':'Indústria de Transformação',
'2949201':'Indústria de Transformação',
'2949299':'Indústria de Transformação',
'2950600':'Indústria de Transformação',
'3011301':'Indústria de Transformação',
'3011302':'Indústria de Transformação',
'3012100':'Indústria de Transformação',
'3031800':'Indústria de Transformação',
'3032600':'Indústria de Transformação',
'3041500':'Indústria de Transformação',
'3042300':'Indústria de Transformação',
'3050400':'Indústria de Transformação',
'3091100':'Indústria de Transformação',
'3091101':'Indústria de Transformação',
'3092000':'Indústria de Transformação',
'3099700':'Indústria de Transformação',
'3101200':'Indústria de Transformação',
'3103900':'Serviços',
'3104700':'Indústria de Transformação',
'3211601':'Indústria de Transformação',
'3211602':'Indústria de Transformação',
'3211603':'Indústria de Transformação',
'3212400':'Indústria de Transformação',
'3220500':'Indústria de Transformação',
'3230200':'Indústria de Transformação',
'3240001':'Indústria de Transformação',
'3240002':'Indústria de Transformação',
'3240003':'Indústria de Transformação',
'3240099':'Indústria de Transformação',
'3250701':'Indústria de Transformação',
'3250702':'Indústria de Transformação',
'3250703':'Indústria de Transformação',
'3250704':'Indústria de Transformação',
'3250705':'Indústria de Transformação',
'3250706':'Indústria de Transformação',
'3250707':'Indústria de Transformação',
'3250708':'Indústria de Transformação',
'3250709':'Indústria de Transformação',
'3291400':'Indústria de Transformação',
'3292201':'Indústria de Transformação',
'3292202':'Indústria de Transformação',
'3299001':'Indústria de Transformação',
'3299002':'Indústria de Transformação',
'3299003':'Indústria de Transformação',
'3299004':'Indústria de Transformação',
'3299005':'Indústria de Transformação',
'3299006':'Indústria de Transformação',
'3299099':'Indústria de Transformação',
'3311200':'Indústria de Transformação',
'3312101':'Indústria de Transformação',
'3312102':'Indústria de Transformação',
'3312103':'Indústria de Transformação',
'3312104':'Indústria de Transformação',
'3313901':'Indústria de Transformação',
'3313902':'Indústria de Transformação',
'3313999':'Serviços',
'3314701':'Indústria de Transformação',
'3314702':'Indústria de Transformação',
'3314703':'Indústria de Transformação',
'3314704':'Indústria de Transformação',
'3314705':'Indústria de Transformação',
'3314706':'Indústria de Transformação',
'3314707':'Indústria de Transformação',
'3314708':'Indústria de Transformação',
'3314709':'Serviços',
'3314710':'Indústria de Transformação',
'3314711':'Indústria de Transformação',
'3314712':'Indústria de Transformação',
'3314713':'Indústria de Transformação',
'3314714':'Indústria de Transformação',
'3316302':'Serviços',
'3314715':'Indústria de Transformação',
'3314716':'Indústria de Transformação',
'3314717':'Indústria de Transformação',
'3314718':'Indústria de Transformação',
'3314719':'Indústria de Transformação',
'3314720':'Indústria de Transformação',
'3314721':'Indústria de Transformação',
'3314722':'Indústria de Transformação',
'3314799':'Indústria de Transformação',
'3315500':'Indústria de Transformação',
'3316301':'Indústria de Transformação',
'4950700':'Serviços',
'3317101':'Indústria de Transformação',
'3319800':'Indústria de Transformação',
'3321000':'Indústria de Transformação',
'3329501':'Indústria de Transformação',
'3317102':'Indústria de Transformação',
'3329599':'Indústria de Transformação',
'3511500':'Serviços Industriais de Utilidade Pública',
'3511501':'Serviços Industriais de Utilidade Pública',
'3511502':'Serviços Industriais de Utilidade Pública',
'3512300':'Serviços Industriais de Utilidade Pública',
'3513100':'Serviços Industriais de Utilidade Pública',
'3514000':'Serviços Industriais de Utilidade Pública',
'3520401':'Serviços Industriais de Utilidade Pública',
'3520402':'Serviços Industriais de Utilidade Pública',
'3530100':'Serviços Industriais de Utilidade Pública',
'3600601':'Serviços Industriais de Utilidade Pública',
'3600602':'Comércio',
'3701100':'Serviços Industriais de Utilidade Pública',
'3702900':'Serviços Industriais de Utilidade Pública',
'3812200':'Serviços Industriais de Utilidade Pública',
'3821100':'Serviços Industriais de Utilidade Pública',
'3822000':'Serviços Industriais de Utilidade Pública',
'3831999':'Indústria de Transformação',
'3831901':'Indústria de Transformação',
'3832700':'Indústria de Transformação',
'3839401':'Serviços Industriais de Utilidade Pública',
'3839499':'Indústria de Transformação',
'3900500':'Serviços Industriais de Utilidade Pública',
'4110700':'Serviços',
'4120400':'Construção Civil',
'4211101':'Construção Civil',
'4211102':'Construção Civil',
'4212000':'Construção Civil',
'4213800':'Construção Civil',
'4221901':'Construção Civil',
'4221902':'Construção Civil',
'4221903':'Construção Civil',
'4221904':'Construção Civil',
'4221905':'Construção Civil',
'4222701':'Construção Civil',
'4223500':'Construção Civil',
'4291000':'Construção Civil',
'4222702':'Construção Civil',
'4292801':'Construção Civil',
'4292802':'Construção Civil',
'4299501':'Construção Civil',
'4299599':'Construção Civil',
'4311801':'Construção Civil',
'4311802':'Construção Civil',
'4312600':'Construção Civil',
'4313400':'Construção Civil',
'4319300':'Construção Civil',
'4321500':'Construção Civil',
'4322301':'Construção Civil',
'4322302':'Construção Civil',
'4322303':'Construção Civil',
'4329101':'Construção Civil',
'4329102':'Construção Civil',
'4329103':'Construção Civil',
'4329104':'Construção Civil',
'4329105':'Construção Civil',
'4329199':'Construção Civil',
'4330401':'Construção Civil',
'4330402':'Construção Civil',
'4330403':'Construção Civil',
'4330404':'Construção Civil',
'4330405':'Construção Civil',
'4330499':'Construção Civil',
'4391600':'Construção Civil',
'4399101':'Construção Civil',
'4399102':'Construção Civil',
'4399103':'Construção Civil',
'4399104':'Construção Civil',
'4399105':'Construção Civil',
'4399199':'Construção Civil',
'4511101':'Comércio',
'4511102':'Comércio',
'4511103':'Comércio',
'8599601':'Serviços',
'4511104':'Comércio',
'4511105':'Comércio',
'4511106':'Comércio',
'4512901':'Comércio',
'4512902':'Comércio',
'4713003':'Comércio',
'4520001':'Comércio',
'4520002':'Comércio',
'4520003':'Comércio',
'4520004':'Comércio',
'4520005':'Comércio',
'4520006':'Comércio',
'4520007':'Comércio',
'4520008':'Comércio',
'4530701':'Comércio',
'4530702':'Comércio',
'4530703':'Comércio',
'4530704':'Comércio',
'4530705':'Comércio',
'4530706':'Comércio',
'4541201':'Comércio',
'4541202':'Comércio',
'4541203':'Comércio',
'4541204':'Comércio',
'4541205':'Comércio',
'4541206':'Comércio',
'4541207':'Comércio',
'4542101':'Comércio',
'4542102':'Comércio',
'4543900':'Comércio',
'4611700':'Comércio',
'4612500':'Comércio',
'4613300':'Comércio',
'4614100':'Comércio',
'5011401':'Serviços',
'4615000':'Comércio',
'4616800':'Comércio',
'4617600':'Comércio',
'4621400':'Comércio',
'4618401':'Comércio',
'4632001':'Comércio',
'4618402':'Comércio',
'4618403':'Comércio',
'4618499':'Comércio',
'4619200':'Comércio',
'4622200':'Comércio',
'4623101':'Comércio',
'4623102':'Comércio',
'4623103':'Comércio',
'4623104':'Comércio',
'4623105':'Comércio',
'4623106':'Comércio',
'4623107':'Comércio',
'4623108':'Comércio',
'4623109':'Comércio',
'4623199':'Comércio',
'4631100':'Comércio',
'4632002':'Comércio',
'4632003':'Comércio',
'4633801':'Comércio',
'4633802':'Comércio',
'4633803':'Comércio',
'4634601':'Comércio',
'4634602':'Comércio',
'4634603':'Comércio',
'4634699':'Comércio',
'4635401':'Comércio',
'4635402':'Comércio',
'4635403':'Comércio',
'4635499':'Comércio',
'8599602':'Serviços',
'4636201':'Comércio',
'4636202':'Comércio',
'4637101':'Comércio',
'4637102':'Comércio',
'4782201':'Comércio',
'4637103':'Comércio',
'4637104':'Comércio',
'4637105':'Comércio',
'4637106':'Comércio',
'4637107':'Comércio',
'4637199':'Comércio',
'4639701':'Comércio',
'4639702':'Comércio',
'4641901':'Comércio',
'4641902':'Comércio',
'4641903':'Comércio',
'4642701':'Comércio',
'4642702':'Comércio',
'4643501':'Comércio',
'4643502':'Comércio',
'4644301':'Comércio',
'4644302':'Comércio',
'4645101':'Comércio',
'4645102':'Comércio',
'4645103':'Comércio',
'4646001':'Comércio',
'4646002':'Comércio',
'4647801':'Comércio',
'4647802':'Comércio',
'5011402':'Serviços',
'4649401':'Comércio',
'4649402':'Comércio',
'4649403':'Comércio',
'4649404':'Comércio',
'4649405':'Comércio',
'4649406':'Comércio',
'4649407':'Comércio',
'4649408':'Comércio',
'4673700':'Comércio',
'4649409':'Comércio',
'4649410':'Comércio',
'4649499':'Comércio',
'4651601':'Comércio',
'4651602':'Comércio',
'4652400':'Comércio',
'4661300':'Comércio',
'4662100':'Comércio',
'4663000':'Comércio',
'4664800':'Comércio',
'4665600':'Comércio',
'4669901':'Comércio',
'4669999':'Comércio',
'4671100':'Comércio',
'4672900':'Comércio',
'4674500':'Comércio',
'4679601':'Comércio',
'4679602':'Comércio',
'4679603':'Comércio',
'4679604':'Comércio',
'4679699':'Comércio',
'4681801':'Comércio',
'4681802':'Comércio',
'4681803':'Comércio',
'4681804':'Comércio',
'4681805':'Comércio',
'4682600':'Comércio',
'4683400':'Comércio',
'4684201':'Comércio',
'4684202':'Comércio',
'4684299':'Comércio',
'4685100':'Comércio',
'4686901':'Comércio',
'4686902':'Comércio',
'4687701':'Comércio',
'4687702':'Comércio',
'4687703':'Comércio',
'4689301':'Comércio',
'4689302':'Comércio',
'4689399':'Comércio',
'4691500':'Comércio',
'4692300':'Comércio',
'4693100':'Comércio',
'4711301':'Comércio',
'4711302':'Comércio',
'4712100':'Comércio',
'4713001':'Comércio',
'4713002':'Comércio',
'6141800':'Serviços',
'4713004':'Comércio',
'4713005':'Comércio',
'4721101':'Comércio',
'4721102':'Comércio',
'4721103':'Comércio',
'4721104':'Comércio',
'4722901':'Comércio',
'4722902':'Comércio',
'4724500':'Comércio',
'4729601':'Comércio',
'4729602':'Comércio',
'4729699':'Comércio',
'4731800':'Comércio',
'4732600':'Comércio',
'4741500':'Comércio',
'4742300':'Comércio',
'4743100':'Comércio',
'4744001':'Comércio',
'4744002':'Comércio',
'4744003':'Comércio',
'4744004':'Comércio',
'4744005':'Comércio',
'4744006':'Comércio',
'4744099':'Comércio',
'4751200':'Comércio',
'4751201':'Comércio',
'4751202':'Comércio',
'4752100':'Comércio',
'4753900':'Comércio',
'4754701':'Comércio',
'4754702':'Comércio',
'4754703':'Comércio',
'4755501':'Comércio',
'4755502':'Comércio',
'4755503':'Comércio',
'4756300':'Comércio',
'4757100':'Comércio',
'4781400':'Comércio',
'4759801':'Comércio',
'5012201':'Serviços',
'4759899':'Comércio',
'4761001':'Comércio',
'4761002':'Comércio',
'4761003':'Comércio',
'4762800':'Comércio',
'4763601':'Comércio',
'4763602':'Comércio',
'4763603':'Comércio',
'4763604':'Comércio',
'4763605':'Comércio',
'4771701':'Comércio',
'4771702':'Comércio',
'4771703':'Comércio',
'4771704':'Comércio',
'4772500':'Comércio',
'4773300':'Comércio',
'4774100':'Comércio',
'4782202':'Comércio',
'4783101':'Comércio',
'4783102':'Comércio',
'4784900':'Comércio',
'4785701':'Comércio',
'4785799':'Comércio',
'4789001':'Comércio',
'4789002':'Comércio',
'4789003':'Comércio',
'4789004':'Comércio',
'4789005':'Comércio',
'4789006':'Comércio',
'4789007':'Comércio',
'4789008':'Comércio',
'4789009':'Comércio',
'4789099':'Comércio',
'4912401':'Serviços',
'4912402':'Serviços',
'4912403':'Serviços',
'4921301':'Serviços',
'4921302':'Serviços',
'4922101':'Serviços',
'4922102':'Serviços',
'4922103':'Serviços',
'4923001':'Serviços',
'4923002':'Serviços',
'4924800':'Serviços',
'4929901':'Serviços',
'4929902':'Serviços',
'4929903':'Serviços',
'4929904':'Serviços',
'4929999':'Serviços',
'4930201':'Serviços',
'4930202':'Serviços',
'4930204':'Serviços',
'5012202':'Serviços',
'5021101':'Serviços',
'5021102':'Serviços',
'5022001':'Serviços',
'5022002':'Serviços',
'5030101':'Serviços',
'5030102':'Serviços',
'5030103':'Serviços',
'5091201':'Serviços',
'5091202':'Serviços',
'5099801':'Serviços',
'5099899':'Serviços',
'5111100':'Serviços',
'5130700':'Serviços',
'5112901':'Serviços',
'5112999':'Serviços',
'5120000':'Serviços',
'5211701':'Serviços',
'5211702':'Serviços',
'5211799':'Serviços',
'5212500':'Serviços',
'5221400':'Serviços',
'5222200':'Serviços',
'5223100':'Serviços',
'5229001':'Serviços',
'5229002':'Comércio',
'5229099':'Serviços',
'5231101':'Serviços',
'5231102':'Serviços',
'5231103':'Serviços',
'5232000':'Serviços',
'5239700':'Serviços',
'5239701':'Serviços',
'5239799':'Serviços',
'5240101':'Serviços',
'5310501':'Serviços',
'5240199':'Serviços',
'5250801':'Serviços',
'5250802':'Serviços',
'5250803':'Serviços',
'5250804':'Serviços',
'5250805':'Serviços',
'5310502':'Serviços',
'5320201':'Serviços',
'5320202':'Serviços',
'5510801':'Serviços',
'5510802':'Serviços',
'5510803':'Serviços',
'5590601':'Serviços',
'5590602':'Serviços',
'5590603':'Serviços',
'5590699':'Serviços',
'5611201':'Serviços',
'7311400':'Serviços',
'5611202':'Serviços',
'5611203':'Serviços',
'5611204':'Serviços',
'5611205':'Serviços',
'5620101':'Indústria de Transformação',
'5620102':'Indústria de Transformação',
'5620103':'Serviços',
'5620104':'Indústria de Transformação',
'5811500':'Indústria de Transformação',
'5812300':'Indústria de Transformação',
'5812301':'Indústria de Transformação',
'5812302':'Indústria de Transformação',
'5813100':'Indústria de Transformação',
'5819100':'Indústria de Transformação',
'5821200':'Indústria de Transformação',
'5822100':'Indústria de Transformação',
'5822101':'Indústria de Transformação',
'5822102':'Indústria de Transformação',
'5823900':'Indústria de Transformação',
'5829800':'Indústria de Transformação',
'5911101':'Indústria de Transformação',
'5911102':'Indústria de Transformação',
'5911199':'Indústria de Transformação',
'5912001':'Indústria de Transformação',
'5912002':'Indústria de Transformação',
'5912099':'Indústria de Transformação',
'5913800':'Serviços',
'5914600':'Serviços',
'6203100':'Serviços',
'5920100':'Indústria de Transformação',
'6021700':'Serviços',
'6022501':'Serviços',
'6022502':'Serviços',
'6110801':'Serviços',
'6110802':'Serviços',
'6110803':'Serviços',
'6110899':'Serviços',
'6120501':'Serviços',
'6120502':'Serviços',
'6120599':'Serviços',
'6130200':'Serviços',
'6142600':'Serviços',
'6143400':'Serviços',
'6190601':'Serviços',
'6190602':'Serviços',
'6190699':'Serviços',
'6201500':'Serviços',
'6201501':'Serviços',
'6201502':'Serviços',
'6202300':'Serviços',
'6911702':'Serviços',
'6204000':'Serviços',
'6209100':'Serviços',
'6311900':'Serviços',
'6319400':'Serviços',
'6391700':'Serviços',
'6399200':'Serviços',
'6410700':'Serviços',
'6421200':'Serviços',
'6422100':'Serviços',
'6423900':'Serviços',
'6424701':'Serviços',
'6424702':'Serviços',
'8599604':'Serviços',
'6424703':'Serviços',
'6424704':'Serviços',
'6431000':'Serviços',
'6432800':'Serviços',
'6433600':'Serviços',
'6434400':'Serviços',
'6435201':'Serviços',
'6435202':'Serviços',
'6435203':'Serviços',
'8593700':'Serviços',
'6436100':'Serviços',
'6437900':'Serviços',
'6438701':'Serviços',
'6438799':'Serviços',
'6440900':'Serviços',
'6450600':'Serviços',
'6461100':'Serviços',
'6462000':'Serviços',
'6463800':'Serviços',
'6470101':'Serviços',
'6470102':'Serviços',
'6470103':'Serviços',
'6491300':'Serviços',
'6492100':'Serviços',
'6499901':'Serviços',
'6499902':'Serviços',
'6499903':'Serviços',
'6499904':'Serviços',
'6499905':'Serviços',
'6499999':'Serviços',
'6511101':'Serviços',
'6511102':'Serviços',
'6512000':'Serviços',
'6520100':'Serviços',
'6530800':'Serviços',
'6541300':'Serviços',
'6542100':'Serviços',
'6550200':'Serviços',
'6611801':'Serviços',
'6611802':'Serviços',
'6611803':'Serviços',
'6611804':'Serviços',
'6612601':'Serviços',
'6612602':'Serviços',
'6612603':'Serviços',
'6612604':'Serviços',
'6612605':'Serviços',
'6613400':'Serviços',
'6619301':'Serviços',
'6619302':'Serviços',
'6619303':'Serviços',
'6619304':'Serviços',
'6619305':'Serviços',
'6619399':'Serviços',
'6621501':'Serviços',
'6621502':'Serviços',
'6622300':'Serviços',
'6629100':'Serviços',
'6630400':'Serviços',
'6810201':'Serviços',
'6810202':'Serviços',
'6810203':'Serviços',
'6821801':'Serviços',
'6821802':'Serviços',
'6822600':'Serviços',
'6911701':'Serviços',
'6911703':'Serviços',
'6912500':'Serviços',
'6920601':'Serviços',
'6920602':'Serviços',
'7020400':'Serviços',
'7111100':'Serviços',
'7112000':'Serviços',
'7119701':'Serviços',
'7119702':'Serviços',
'7119703':'Serviços',
'7119704':'Serviços',
'7119799':'Serviços',
'7120100':'Serviços',
'7210000':'Serviços',
'7220700':'Serviços',
'7312200':'Serviços',
'7319001':'Serviços',
'7319002':'Serviços',
'7319003':'Serviços',
'7319004':'Serviços',
'7319099':'Serviços',
'7320300':'Serviços',
'7410201':'Serviços',
'7410202':'Serviços',
'7410203':'Serviços',
'7410299':'Serviços',
'7420001':'Serviços',
'7420002':'Serviços',
'7420003':'Serviços',
'7420004':'Serviços',
'7420005':'Serviços',
'7490101':'Serviços',
'7490102':'Serviços',
'7490103':'Serviços',
'7490104':'Serviços',
'7490105':'Serviços',
'8599605':'Serviços',
'7500100':'Serviços',
'7490199':'Serviços',
'7711000':'Serviços',
'7719501':'Serviços',
'7719502':'Serviços',
'7719599':'Serviços',
'7721700':'Serviços',
'7722500':'Serviços',
'7723300':'Serviços',
'7729201':'Serviços',
'7729202':'Serviços',
'7729203':'Serviços',
'7729299':'Serviços',
'7731400':'Serviços',
'7732201':'Serviços',
'7732202':'Serviços',
'7733100':'Serviços',
'7739001':'Serviços',
'7739002':'Serviços',
'8599699':'Serviços',
'7739003':'Serviços',
'7739099':'Serviços',
'7740300':'Serviços',
'7810800':'Serviços',
'7820500':'Serviços',
'7830200':'Serviços',
'7911200':'Serviços',
'7912100':'Serviços',
'7990200':'Serviços',
'8011101':'Serviços',
'8011102':'Serviços',
'8012900':'Serviços',
'8020000':'Serviços',
'8020001':'Serviços',
'8020002':'Serviços',
'8030700':'Serviços',
'8111700':'Serviços',
'8112500':'Serviços',
'8121400':'Serviços',
'8122200':'Serviços',
'8129000':'Serviços',
'8130300':'Agropecuária, Extração Vegetal, Caça e Pesca',
'8211300':'Serviços',
'8219901':'Serviços',
'8219999':'Serviços',
'8220200':'Serviços',
'8230001':'Serviços',
'8230002':'Serviços',
'8291100':'Serviços',
'8292000':'Serviços',
'8299701':'Serviços',
'8299702':'Serviços',
'8299703':'Serviços',
'8299704':'Serviços',
'8299705':'Serviços',
'8299706':'Serviços',
'8299707':'Serviços',
'8299799':'Serviços',
'8411600':'Administração Pública',
'8412400':'Administração Pública',
'8413200':'Administração Pública',
'8421300':'Administração Pública',
'8422100':'Administração Pública',
'8423000':'Administração Pública',
'8424800':'Administração Pública',
'8425600':'Administração Pública',
'8430200':'Serviços',
'8511200':'Serviços',
'8512100':'Serviços',
'8513900':'Serviços',
'8520100':'Serviços',
'8531700':'Serviços',
'8532500':'Serviços',
'8533300':'Serviços',
'8541400':'Serviços',
'8542200':'Serviços',
'8550301':'Serviços',
'8550302':'Serviços',
'8591100':'Serviços',
'8592901':'Serviços',
'8592902':'Serviços',
'8592903':'Serviços',
'8592999':'Serviços',
'8610101':'Serviços',
'8610102':'Serviços',
'8621601':'Serviços',
'8621602':'Serviços',
'8622400':'Serviços',
'8630501':'Serviços',
'8630502':'Serviços',
'8630503':'Serviços',
'8630504':'Serviços',
'8630505':'Serviços',
'8630506':'Serviços',
'8630507':'Serviços',
'8630599':'Serviços',
'8640201':'Serviços',
'8640202':'Serviços',
'8640203':'Serviços',
'8640204':'Serviços',
'8640205':'Serviços',
'8640206':'Serviços',
'8640207':'Serviços',
'8640208':'Serviços',
'8640209':'Serviços',
'8640210':'Serviços',
'8640211':'Serviços',
'8640212':'Serviços',
'8640213':'Serviços',
'8640214':'Serviços',
'8640299':'Serviços',
'8650001':'Serviços',
'8650002':'Serviços',
'8650003':'Serviços',
'8650004':'Serviços',
'8650005':'Serviços',
'8650006':'Serviços',
'9491000':'Serviços',
'8650099':'Serviços',
'8660700':'Serviços',
'8690901':'Serviços',
'8690902':'Serviços',
'8650007':'Serviços',
'8690903':'Serviços',
'8690904':'Serviços',
'8690999':'Serviços',
'8711501':'Serviços',
'8711502':'Serviços',
'8711503':'Serviços',
'8711504':'Serviços',
'8711505':'Serviços',
'8712300':'Serviços',
'8720401':'Serviços',
'8720499':'Serviços',
'8730101':'Serviços',
'8730102':'Serviços',
'8730199':'Serviços',
'8800600':'Serviços',
'9001901':'Serviços',
'9001902':'Serviços',
'9001903':'Serviços',
'9001904':'Serviços',
'9001905':'Serviços',
'9001906':'Serviços',
'9001999':'Serviços',
'9002701':'Serviços',
'9002702':'Serviços',
'9003500':'Serviços',
'9101500':'Serviços',
'9102301':'Serviços',
'9102302':'Serviços',
'9103100':'Serviços',
'9200301':'Serviços',
'9200302':'Serviços',
'9200399':'Serviços',
'9311500':'Serviços',
'9312300':'Serviços',
'9313100':'Serviços',
'9319199':'Serviços',
'9321200':'Serviços',
'9329801':'Serviços',
'9329802':'Serviços',
'9329803':'Serviços',
'9329804':'Serviços',
'9329899':'Serviços',
'9411100':'Serviços',
'9412000':'Serviços',
'9412001':'Serviços',
'9412099':'Serviços',
'9420100':'Serviços',
'9430800':'Serviços',
'9492800':'Serviços',
'9493600':'Serviços',
'9499500':'Serviços',
'9601701':'Serviços',
'9511800':'Serviços',
'9512600':'Comércio',
'9521500':'Comércio',
'9529101':'Comércio',
'9529102':'Comércio',
'9529103':'Comércio',
'9529104':'Comércio',
'9529105':'Comércio',
'9529106':'Comércio',
'9529199':'Comércio',
'9601703':'Serviços',
'9602501':'Serviços',
'9602502':'Serviços',
'9603302':'Serviços',
'9603303':'Serviços',
'9603304':'Serviços',
'9603305':'Serviços',
'0111301':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0111302':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0111303':'Agropecuária, Extração Vegetal, Caça e Pesca',
'0111399':'Agropecuária, Extração Vegetal, Caça e Pesca',
'9603399':'Serviços',
'9609201':'Serviços',
'9609202':'Serviços',
'9609203':'Serviços',
'9609204':'Serviços',
'1051100':'Indústria de Transformação',
'9319101':'Serviços',
'9601702':'Serviços',
'9609205':'Serviços',
'9609206':'Serviços',
'9609207':'Serviços',
'9609208':'Serviços',
'9609299':'Serviços',
'9900800':'Serviços',
'9999999':'-1',
'9999997': '-1',
'-1':'-1'
}

In [12]:
# Dicionário para renomeação de colunas
dict_columns = {
"BAIRRO_SP"                : "bairro_sp",
"BAIRRO FORT"              : "bairro_fortaleza",
"BAIRRO_RJ"                : "bairro_rj",
"CLAS CNAE 20"             : "cnae_2_classe",
"CLAS CNAE 95"             : "cnae_1_classe",
"DISTRITO_SP"              : "distrito_sp",
"EST CLT OUT"              : "qtd_vinc_clt",
"ESTOQUE"                  : "qtd_vinc_ativos",
"ESTOQUE ESTA"             : "qtd_vinc_estat",
"IND ATIV ANO"             : "ind_ativ_ano",
"IND CEI VINC"             : "ind_cei_vinculado",
"IND PAT"                  : "ind_pat",
"IND RAIS NEG"             : "ind_rais_negativa",
"IND SIMPLES"              : "ind_simples",
"MUNICIPIO"                : "id_municipio",
"NAT JURIDICA"             : "nat_jur",
"REG ADM DF"               : "regiao_adm_df",
"SB CLAS 20"               : "cnae_2_subclasse",
"TAMESTAB"                 : "tam_estab",
"TIPO ESTBL"               : "tipo_estab",
"UF"                       : "sigla_uf",
"IBGE Subsetor"            : "subsetor_ibge",
"Bairros SP"               : "bairro_sp",
"Bairros Fortaleza"        : "bairro_fortaleza",
"Bairros RJ"               : "bairro_rj",
"CNAE 2.0 Classe"          : "cnae_2_classe",
"CNAE 95 Classe"           : "cnae_1_classe",
"Distritos SP"             : "distrito_sp",
"Qtd Vínculos CLT"         : "qtd_vinc_clt",
"Qtd Vínculos Ativos"      : "qtd_vinc_ativos",
"Qtd Vínculos Estatutários": "qtd_vinc_estat",
"Ind Atividade Ano"        : "ind_ativ_ano",
"Ind CEI Vinculado"        : "ind_cei_vinculado",
"Ind Estab Participa PAT"  : "ind_pat",
"Ind Rais Negativa"        : "ind_rais_negativa",
"Ind Simples"              : "ind_simples",
"Município"                : "id_municipio",
"Natureza Jurídica"        : "nat_jur",
"Regiões Adm DF"           : "regiao_adm_df",
"CNAE 2.0 Subclasse"       : "cnae_2_subclasse",
"Tamanho Estabelecimento"  : "tam_estab",
"Tipo Estab"               : "tipo_estab",
"Tipo Estab.1"             : "ctg_tipo_estab",
"Tipo Estab19"             : "tipo_estab",
"Tipo Estab20"             : "ctg_tipo_estab",
"UF"                       : "sigla_uf",
"IBGE Subsetor"            : "subsetor_ibge",
"CEP Estab"                : "cep",
"CEP Estb"                 : "cep" 
}

In [13]:
def tratamento_rais_estab_pyspark(caminho_arquivo, ano, ds_owner):
    # Iniciando uma sessão Spark
    # com:
    # alta memória por executor
    # alto número de núcleos por executor
    # alto número de executores 
    # alta memória do driver
    # alto tamanho da partição para leitura de dados do MongoDB

    spark = SparkSession.builder \
    .appName("Tratando RAIS estab com PySpark")\
        .config("spark.executor.memory", "8g")\
        .config("spark.executor.cores", "4")\
        .config("spark.executor.instances", "4")\
        .config("spark.driver.memory", "4g")\
        .config("spark.mongodb.input.partitionerOptions.partitionSizeMB", "128")\
        .config("spark.network.timeout", "10000")\
        .getOrCreate()

    # Leitura do arquivo CSV
    df = spark.read.option("delimiter", ";") \
        .option("encoding", "cp1252") \
        .csv(caminho_arquivo, header=True)

    # Renomeando colunas via dicionário
    for old_col, new_col in dict_columns.items():
        df = df.withColumnRenamed(old_col, new_col)

    # Criando colunas
    df = df.withColumn("ano", lit(int(ano))) \
        .withColumn("curr_date", lit(str(date.today()))) \
        .withColumn("ds_owner", lit(ds_owner)) \
        .withColumn("sigla_uf", expr("id_municipio / 10000").cast("int"))

    if int(ano) < 2014:
        df = df.withColumn("cep", lit(-1)) \
            .withColumn("subsetor_ibge", lit(-1))
    else:
        df = df.drop("ctg_tipo_estab")

    # Removendo espaços no início e final dos campos
    for col_name in df.columns:
        if df.schema[col_name].dataType == "string":
            df = df.withColumn(col_name, trim(col(col_name)))

    # Substituindo valores não identificados por -1
    replace_values = {"0000-1": "-1",
                    "000-1": "-1",
                    "00-1": "-1",
                    "0-1": "-1",
                    "{ñ": "-1",
                    "{ñ class}": "-1",
                    "{ñ c": "-1",
                    "{ñ clas": "-1",
                    "{ñ cl": "-1"}

    for old_val, new_val in replace_values.items():
        df = df.replace(old_val, new_val)

    # Substituindo nulos por -1
    cols_to_replace_nulls = ["bairro_sp", "bairro_fortaleza", "bairro_rj", "cnae_2_classe", "cnae_1_classe",
                            "distrito_sp", "qtd_vinc_clt", "qtd_vinc_ativos", "qtd_vinc_estat", "ind_ativ_ano",
                            "ind_cei_vinculado", "ind_pat", "ind_rais_negativa", "ind_simples", "id_municipio",
                            "nat_jur", "regiao_adm_df", "cnae_2_subclasse", "tam_estab", "tipo_estab", "sigla_uf",
                            "subsetor_ibge", "cep"]

    for col_name in cols_to_replace_nulls:
        df = df.withColumn(col_name, when(col(col_name).isNull(), -1).otherwise(col(col_name)))

    
    ############PADRONIZAÇÕES DO SEBRAE######################
    
    # 1. CRIANDO PADRONIZAÇÃO DO QUE É EMPRESA PRO SEBRAE
        

    # Verificando length da coluna de cnae_2_subclasse porque numeros inteiros perdem o zero a esquerda necessário para a padronização
    df = df.withColumn("subc_len", length(col("cnae_2_subclasse")))

    # Criando coluna subc_comp para complementar coluna de cnae quando ela tem 6 dígitos
    conditions = [
        (col("subc_len") == 6),
        (col("subc_len") >= 7)
    ]
    values = ['0', '']
    df = df.withColumn("subc_comp", when(conditions[0], values[0]).otherwise(values[1]))

    # Concatenando 'subc_comp' com 'cnae_2_subclasse'
    df = df.withColumn("cnae_2_subclasse", concat_ws("", col("subc_comp"), col("cnae_2_subclasse")))

    # Dropando colunas temporárias
    df = df.drop("subc_len", "subc_comp")

    # Convertendo 'nat_jur' para string
    df = df.withColumn("nat_jur", col("nat_jur").cast("string"))

    # Criando coluna com dois primeiros digitos da CNAE (adicionando 0 a esquerda, se necessário)
    df = df.withColumn("cnae_2_subc_2_dig", lpad(col("cnae_2_subclasse").cast("int").cast("string"), 2, "0"))

    # Definindo a condição para 'padrao_sebrae'
    condicao = (
        (~col("cnae_2_subc_2_dig").isin(['84', '94', '99'])) &
        col("nat_jur").isin(['2046', '2054', '2062', '2135', '2232', '2240', '2259', '2267', 
                            '2283', '2291', '2305', '2313', '2321', '2330', '2348', '4014', 
                            '4022', '4081', '4111', '4120'])
    )

    # Criando a coluna 'padrao_sebrae' com base na condição anterior
    df = df.withColumn("padrao_sebrae", when(condicao, 1).otherwise(0))

   
   # 2. CRIANDO PADRONIZAÇÃO DE PORTE DO SEBRAE

    @udf(StringType())
    def format_tam_estab(tam_estab):
        return str(int(tam_estab))

    df = df.withColumn("tam_estab", format_tam_estab(col("tam_estab")))

    # Criando função para mapear cnaes e retornar setor ibge com base no dicionário
    @udf(StringType())
    def map_cnae_to_setor(cnae_2_subc):
        return dpara_cnae2_setoribge.get(cnae_2_subc, "-1")

    # Aplicando a função
    df = df.withColumn("setor_ibge", map_cnae_to_setor(df["cnae_2_subclasse"]))

    # Concatenando setor_ibge e tam_estab
    df = df.withColumn("concat", concat_ws("", col("setor_ibge"), col("tam_estab")))

    # Criando função para mapear as concatençaões e retornar o porte sebrae com base no dicionário
    @udf(StringType())
    def map_porte_sebrae(concat):
        return pad_sebrae_porte.get(concat, "-1")
    
    # Aplicando a função
    df = df.withColumn("porte_sebrae", map_porte_sebrae(df["concat"]))


    # 3. CRIANDO PADRONIZAÇÃO DE SETOR DO SEBRAE

    # Criando função para mapear as cnaes e retornar o setor sebrae com base no dicionário
    @udf(StringType())
    def map_setor_sebrae(cnae_2_subc):
        return pad_sebrae_setor.get(cnae_2_subc, "-1")

    # Aplicando a função
    df = df.withColumn('setor_sebrae', map_setor_sebrae(df['cnae_2_subclasse']))


    ########## FINALIZANDO TRATAMENTOS ##########

    # Conversão para int das colunas int
    colunas_para_int = ["bairro_sp", "bairro_fortaleza", "bairro_rj", "cnae_2_classe", "cnae_1_classe", "distrito_sp", 
                        "qtd_vinc_clt", "qtd_vinc_ativos", "qtd_vinc_estat", "ind_ativ_ano", "ind_cei_vinculado", 
                        "ind_pat", "ind_rais_negativa", "ind_simples", "id_municipio", "nat_jur", "regiao_adm_df",
                        "cnae_2_subclasse", "tam_estab", "tipo_estab", "sigla_uf", "subsetor_ibge", "cep", "padrao_sebrae"
                        ]

    for coluna in colunas_para_int:
        df = df.withColumn(coluna, col(coluna).cast("int"))

    # Reordenando colunas
    df = df.select("ds_owner", "curr_date", 
                "ano", "sigla_uf", "id_municipio", "qtd_vinc_ativos", "qtd_vinc_clt", "qtd_vinc_estat",
                "nat_jur", "tam_estab", "tipo_estab",
                "ind_ativ_ano", "ind_cei_vinculado", "ind_pat", "ind_rais_negativa", "ind_simples",
                "cnae_1_classe", "cnae_2_classe", "cnae_2_subclasse", "subsetor_ibge",
                "cep", "bairro_sp", "bairro_fortaleza", "bairro_rj", "distrito_sp", "regiao_adm_df",
                "padrao_sebrae", "porte_sebrae", "setor_sebrae", "setor_ibge")

    # Preenchendo valores nulos e removendo espaços
    for col_name in df.columns:
        if df.schema[col_name].dataType == "integer":
            df = df.fillna({col_name: -1})
        elif df.schema[col_name].dataType == "string":
            df = df.fillna({col_name: "-1"})
            df = df.withColumn(col_name, trim(col(col_name)))
    
    print(f"Show, tratamento básico dos dados de {ano} finalizados...")
    return df

In [14]:
# Função para remover arquivos .txt
def remover_arquivo_txt(caminho_arquivo):
    if caminho_arquivo.endswith(".txt"):
        os.remove(caminho_arquivo)
        print(f"\nArquivo txt removido do {caminho_arquivo} com sucesso!\n")

In [15]:
def gerar_reports_ae_spark(df, ano):
    qt_estabel = df.count()
    qt_vinc = df.select(sum("qtd_vinc_ativos")).collect()[0][0]

    print(f"O número de estabelecimentos declarando RAIS em {ano} é: {qt_estabel}.")
    print(f"O número total de vínculos empregatícios ativos declarados em {ano} é: {qt_vinc}.")

    print("\nRelação de estabelecimentos e vínculos por UF:")
    df.groupby("sigla_uf").agg({"qtd_vinc_ativos": "sum", "id_municipio": "count"}).show()

    lista = ['padrao_sebrae', 'porte_sebrae', 'setor_sebrae', 'setor_ibge']
    for col in lista:
        null_count = df.filter(col(col).isNull()).count()
        print(f'\nRelatório de nulos na coluna "{col}" para o ano de {ano}: {null_count}')

    print(f'\nRelatório de valores -1 nas colunas para o ano de {ano}:\n')
    for col in df.columns:
        if df.schema[col].dataType == IntegerType():
            neg_one_count = df.filter(col(col) == -1).count()
            print(f'{col}: {neg_one_count}')

In [16]:
def load_config(file_path=r'..\..\config.json'):
    with open(file_path, 'r') as file:
        config = json.load(file)
    return config

In [17]:
def upload_mongodb(df, arquivo, ano, batch_size=1000):
    print("------- Conectando com o MongoDB ------------------")

    config = load_config()

    host = config["mongodb"]["host"]
    port = config["mongodb"]["port"]
    user_name = config["mongodb"]["user_name"]
    password = config["mongodb"]["password"]
    db_name = config["mongodb"]["db_name"]

    collection_name = arquivo

    spark = SparkSession.builder \
        .appName("Upload para MongoDB") \
        .config("spark.mongodb.output.uri", f"mongodb://{user_name}:{password}@{host}:{port}/{db_name}.{collection_name}") \
        .config("spark.executor.memory", "8g")\
        .config("spark.executor.cores", "4")\
        .config("spark.executor.instances", "4")\
        .config("spark.driver.memory", "4g")\
        .config("spark.mongodb.input.partitionerOptions.partitionSizeMB", "128")\
        .config("spark.network.timeout", "10000")\
        .getOrCreate()

    df.write.format("com.mongodb.spark.sql.DefaultSource") \
        .mode("append") \
        .option("uri", f"mongodb://{user_name}:{password}@{host}:{port}/{db_name}.{collection_name}") \
        .option("batchsize", batch_size) \
        .save()

    print(f"{arquivo} de {ano} gravado com sucesso!")
    print("\n------- Fim da carga dos dados para o MongoDB ------------------\n")


In [18]:
def upload_postgresql(df, batch_size=1000):
    print("Iniciando a transferência para o PostgreSQL...")

    config = load_config()

    pg_url = config["postgresql"]["pg_url"]
    pg_table = config["postgresql"]["pg_table"]
    pg_user = config["postgresql"]["pg_user"]
    pg_password = config["postgresql"]["pg_password"]

    spark = SparkSession.builder \
        .appName("Transferência para PostgreSQL") \
        .getOrCreate()

    df.write.format("jdbc") \
        .option("url", pg_url) \
        .option("dbtable", pg_table) \
        .option("user", pg_user) \
        .option("password", pg_password) \
        .option("batchsize", batch_size) \
        .mode("append") \
        .save()

    print("Transferência de dados concluída.")

# **5. RODANDO**:

In [ ]:
ds_owner = input("Digite seu 1º e 2º nome, ex: Marcilio Duarte")
inicio = datetime.now()
print("\nO código começou a rodar em:\n", inicio, "\n")
for ano in anos:
    print(f"Começando o loop para o ano de {ano}...\n")
    # Portal principal da RAIS
    ftp = FTP("ftp.mtps.gov.br")  ## SERVIDOR FTP
    link = r"ftp://ftp.mtps.gov.br/pdet/microdados/RAIS/"
    # Comando que conecta a aba correta dentro do site ftp
    cmd = r"pdet/microdados/RAIS/" 
    ftp.login()
    # Navegando até o diretório do ano
    ftp.cwd(cmd + str(ano))
    
    # Obtendo a lista de arquivos no diretório do ano
    arquivos = ftp.nlst()
    for arquivo in arquivos:
        try:
            if (arquivo=="RAIS_ESTAB_PUB.7z") | (arquivo.startswith("ESTB")) | (arquivo.startswith("Estb")):
                # Checando se o arquivo já existe no diretório local
                if not checadir(base, arquivo):
                    # Baixando o arquivo
                    baixar_arquivo(ftp, arquivo, base)
                    
                    # Extraindo o arquivo
                    caminho_arquivo = os.path.join(base, arquivo)
                    extrair_arquivo_7z_txt(caminho_arquivo)
                    
                    # Removendo o arquivo zipado
                    remover_arquivo_7z(caminho_arquivo)
                    
                    ## Tratando o dataframe
                    if ano != 2022:
                        caminho_arquivo = caminho_arquivo[:-2] + "txt"
                        df = tratamento_rais_estab_pyspark(caminho_arquivo, ano, ds_owner=ds_owner)
                    elif ano == 2022:
                        try:
                            caminho_arquivo = caminho_arquivo[:-2] + "txt.txt"
                            df = tratamento_rais_estab_pyspark(caminho_arquivo, ano, ds_owner=ds_owner)
                        except Exception as e:
                            print(f"Erro na leitura: {e}. Vamos tentar ler conforme os anos anteriores a 2022...")
                            caminho_arquivo = caminho_arquivo[:-4]
                            df = tratamento_rais_estab_pyspark(caminho_arquivo, ano, ds_owner=ds_owner)

                    # Gerando reports de AE
                    #gerar_reports_ae_spark(df, str(ano))

                    # Importando para mongodb
                    
                    try:                   
                        upload_mongodb(df, arquivo="RAIS_ESTAB_PUB", ano=str(ano))
                    except Exception as e:
                        print(f"Erro: {e}, tentando novamente...")
                        upload_mongodb(df, arquivo="RAIS_ESTAB_PUB", ano=str(ano))          

                    try:                   
                        upload_postgresql(df = df, batch_size=1000)
                    except Exception as e:
                        print(f"Erro: {e}, tentando novamente...")
                        upload_postgresql(df = df, batch_size=1000)    

                    # Removendo o arquivo .txt
                    remover_arquivo_txt(caminho_arquivo)
                    
                else:
                    print(f"O arquivo {arquivo} já existe na pasta. Indo para o próximo!\n")
            else:
                print(f"Não iremos trabalhar com o arquivo {arquivo}...")
        except Exception as e:
            print(f"Um erro ocorreu na execução do ano {str(ano)} e no arquivo {arquivo}. Iremos continuar a partir dos próximos anos, mas o erro em questão foi: {e}")
            continue
        
    # Voltando ao diretório raiz para a próxima iteração
    continue
fim = datetime.now()
print("\nO código terminou de rodar em:\n", fim, "\n")
print("\nO tempo total foi:\n", fim - inicio, "\n") 